# **Loading libraries**



In [ ]:
%pip install gdown
%pip install tqdm scikit-learn
%pip install geopandas
%pip install geohash2
%pip install folium
%pip install python-geohash



import pandas as pd
import plotly.express as px
import gdown
import os
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde
from google.colab import drive
from math import radians, sin, cos, sqrt, atan2
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
from tqdm import tqdm
import geohash2
from sklearn.cluster import KMeans
import json
import folium
from folium.plugins import MarkerCluster
from scipy.stats import chi2_contingency
import geohash as gh







In [ ]:
drive.mount('/content/drive')


# **Loading the data**



---
downloading the dataset
---



In [ ]:
folder_id = '1O3w5OKnS__hzlL8kTSfGCUc_iX8XNjEN'
output_dir = 'Homework'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print(f"Attempting to download content from folder ID: {folder_id} into {output_dir}")
try:
    gdown.download_folder(id=folder_id, output=output_dir, quiet=False, use_cookies=False)
    print(f"\nSuccessfully downloaded content to: /content/{output_dir}")
    print("You can now find the downloaded content in the 'downloaded_external_folder' directory in your Colab files browser.")
except Exception as e:
    print(f"\nAn error occurred during download: {e}")
    print("Please ensure the Google Drive folder is publicly accessible or shared with 'Anyone with the link can view'.")

stations_info=pd.read_csv("Homework/data/Capital_Bikeshare_Locations.csv")
#
# Load tabular data
weather_df = pd.read_csv("Homework/data/Washington,DC,USA 2024-01-01 to 2024-12-31.csv")
trips_df = pd.read_parquet('Homework/data/daily-rent.parquet')

# Load spatial parking zones
parking_zones_gdf = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')

stations_df = pd.read_csv("Homework/data/Capital_Bikeshare_Locations.csv")
# Load spatial parking zones
parking_zones_gdf = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')
Shuttle_Bus_Stops=pd.read_csv("Homework/data/Shuttle_Bus_Stops.csv")
Metro_Bus_Stops =pd.read_csv("Homework/data/Metro_Bus_Stops.csv")
#Loading Residential and Visitor Parking Zones
Residential_Visitor_Parking_Zones  = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')






---
Downloading the combined and modified dataset (for ease of use )
---



In [ ]:
file_id = "114g7JYuZ00i864przAIJQYymib_5h6Qa"  # Replace with your actual file ID
output_file = "trips_df.csv"  # You can change the output file name

gdown.download(id=file_id, output=output_file, quiet=False)
trips_df = pd.read_csv(output_file)
print(f"File downloaded to {output_file}")


# **Preprocessing , Cleaning & inspecting the data**


There is a problem with missing start/id , almost 20% of the data are nulls so we must find a way to fill these up

**spatial join**


---


using lang and lati we can match it to the nearest station and then assign this id

In [ ]:
trips_df = trips_df.dropna(subset=['end_lat', 'end_lng'])

trips_df_cleaned=trips_df.drop_duplicates()
trips_df_cleaned.isna().sum()

In [ ]:
# EPSG:4326 = lat/lon
trips_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['start_lng'], trips_df['start_lat']),
    crs='EPSG:4326'
)

stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df['LONGITUDE'], stations_df['LATITUDE']),
    crs='EPSG:4326'
)


In [ ]:
# Find nearest station to each ride
trips_with_nearest_station = gpd.sjoin_nearest(
    trips_gdf, stations_gdf[['STATION_ID', 'geometry']],
    how="left", distance_col="distance"
)

# Now we fill missing station_id with nearest one
trips_df['start_station_id'] = trips_df['start_station_id'].fillna(
    trips_with_nearest_station['STATION_ID']
)
# Creating a mapping from STATION_ID to STATION_NAME
id_to_name = stations_df.set_index('STATION_ID')['NAME'].to_dict()

# Fill in missing start_station_name using start_station_id
trips_df['start_station_name'] = trips_df['start_station_name'].fillna(
    trips_df['start_station_id'].map(id_to_name)
)
trips_df_cleaned=trips_df.drop_duplicates()
trips_df_cleaned.isna().sum()

Repeating the process to end id and name

In [ ]:
trips_gdf_end = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs='EPSG:4326'
)

trips_with_nearest_end_station = gpd.sjoin_nearest(
    trips_gdf_end, stations_gdf[['STATION_ID', 'geometry']],
    how="left", distance_col="end_distance"
)

trips_df['end_station_id'] = trips_df['end_station_id'].fillna(
    trips_with_nearest_end_station['STATION_ID']
)
trips_df['end_station_name'] = trips_df['end_station_name'].fillna(
    trips_df['end_station_id'].map(id_to_name)
)
trips_df=trips_df.drop_duplicates()
trips_df.isna().sum()

we will continue inspecting the rest of the data

In [ ]:
stations_df=stations_df.drop_duplicates()
stations_df.isna().sum()  # we dont need to drop null values

In [ ]:
weather_df=weather_df.drop_duplicates()
weather_df.isna().sum()

In [ ]:
parking_zones_gdf.head(2)

In [ ]:
parking_zones_gdf=parking_zones_gdf.drop_duplicates()
parking_zones_gdf.isna().sum()

In [ ]:
parking_zones_gdf = parking_zones_gdf.drop(columns=['CREATOR', 'CREATED','EDITOR','EDITED'])


In [ ]:
parking_zones_gdf=parking_zones_gdf.drop_duplicates()
parking_zones_gdf.isna().sum()

---
**The outside WDC problem :**

In [ ]:
# The bounding box method
DC_LAT_MIN = 38.7916
DC_LAT_MAX = 38.9955
DC_LNG_MIN = -77.1198
DC_LNG_MAX = -76.9094
# Filtering  Points Outside the Bounding Box
out_of_bounds_start = ~(
    (trips_df['start_lat'].between(DC_LAT_MIN, DC_LAT_MAX)) &
    (trips_df['start_lng'].between(DC_LNG_MIN, DC_LNG_MAX))
)

out_of_bounds_end = ~(
    (trips_df['end_lat'].between(DC_LAT_MIN, DC_LAT_MAX)) &
    (trips_df['end_lng'].between(DC_LNG_MIN, DC_LNG_MAX))
)

# Combine both to detect any trip with at least one bad coordinate
outlier_mask = out_of_bounds_start | out_of_bounds_end
outliers = trips_df[outlier_mask]

# Inspect the Outliers
print(f"Number of outlier trips: {len(outliers)}")
outliers[['ride_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']].head()
# Total number of trips in the dataset
total_trips = len(trips_df)

# Number of outliers detected
num_outliers = len(outliers)

# Calculate the percentage of outliers
percentage_outliers = (num_outliers / total_trips) * 100

print(f"Percentage of outlier trips: {percentage_outliers:.2f}%")


In [ ]:
# we will drop them
trips_df = trips_df[~outlier_mask].reset_index(drop=True)


In [ ]:
# checking ride_id
print("Duplicate ride_ids:", trips_df['ride_id'].duplicated().sum())
print("Missing ride_ids:", trips_df['ride_id'].isna().sum())


*there an issue with dublicated ride_id so we will only keep the first occurrence*

In [ ]:
# Keep first occurrence or drop based on your context:
trips_df = trips_df.drop_duplicates(subset='ride_id', keep='first')

In [ ]:
print("Null times:", trips_df[['started_at', 'ended_at']].isna().sum())
print("Negative durations:", (trips_df['ended_at'] < trips_df['started_at']).sum())


In [ ]:
print("Missing start station:", trips_df['start_station_id'].isna().sum())
print("Missing end station:", trips_df['end_station_id'].isna().sum())


In [ ]:
zero_coords = trips_df[(trips_df['start_lat'] == 0) | (trips_df['start_lng'] == 0)]
print("Zero coordinates:", len(zero_coords))

In [ ]:
# checkign if rideable_type and member_casual has weird values
print("Ride types:", trips_df['rideable_type'].unique())
print("Member types:", trips_df['member_casual'].unique())


In [ ]:
weather_df.head(1)

In [ ]:
# first we make sure all the dates are in the same format (by checking the length)
datetime_lengths = weather_df["datetime"].astype(str).apply(len)
print(datetime_lengths.value_counts())
weather_df["date"] = pd.to_datetime(weather_df["datetime"])
print(weather_df["date"].dtype)


In [ ]:
trips_df["start_time"] = pd.to_datetime(trips_df["started_at"])
trips_df["end_time"] = pd.to_datetime(trips_df["ended_at"])
# ensuring that CRS is EPSG:4326
if parking_zones_gdf.crs != "EPSG:4326":
    parking_zones_gdf = parking_zones_gdf.to_crs("EPSG:4326")
# Spatial Join to Map Stations to Parking Zones
# Spatial join: add zone info to each station
stations_with_zone = gpd.sjoin(
    stations_gdf,
    parking_zones_gdf[["NAME", "geometry"]],
    how="left",
    predicate="within"
)
# Rename column for clarity
stations_with_zone = stations_with_zone.rename(columns={"zone_name": "residential_zone"})
# Joining Weather Data
# Extract date from start_time for weather join
trips_df["date"] = trips_df["start_time"].dt.date
weather_df["date"] = weather_df["date"].dt.date

# Join weather by date
trips_df = trips_df.merge(weather_df, on="date", how="left")



In [ ]:
trips_df[['start_station_id', 'end_station_id', 'start_station_name', 'end_station_name']].isnull().sum()


In [ ]:
trips_df.head(2)



---

# **Feature engineering**

---




---
B1
---


In [ ]:
# B1

# From started_at
trips_df['start_year'] = trips_df['started_at'].dt.year
trips_df['start_month'] = trips_df['started_at'].dt.month
trips_df['start_day_num'] = trips_df['started_at'].dt.day
trips_df['start_day_name'] = trips_df['started_at'].dt.day_name()

# From ended_at
trips_df['end_year'] = trips_df['ended_at'].dt.year
trips_df['end_month'] = trips_df['ended_at'].dt.month
trips_df['end_day_num'] = trips_df['ended_at'].dt.day
trips_df['end_day_name'] = trips_df['ended_at'].dt.day_name()
trips_df.head(2)



---
B2
---


In [ ]:
trips_df['trip_duration_minutes'] = (trips_df['end_time'] - trips_df['start_time']).dt.total_seconds() / 60
trips_df['trip_duration_minutes']=trips_df['trip_duration_minutes'].round(2)
trips_df['trip_duration_minutes'].head(3)

**The trip_duration_minutes problem**

In [ ]:
trips_df['trip_duration_minutes'].describe()

*we can clearly see that there is a problem with the tripd_durations, the min is a negative value and that is not right*

In [ ]:
# Show trips with negative or 0 duration
invalid_durations = trips_df[trips_df['trip_duration_minutes'] <= 0]
print(f"Invalid rows: {len(invalid_durations)}")
invalid_durations[['ride_id', 'started_at', 'ended_at', 'trip_duration_minutes']].head()


In [ ]:
# Filter only valid trips
trips_df = trips_df[trips_df['trip_duration_minutes'] > 0]
trips_df['trip_duration_minutes'].describe()


---
B3
---


In [ ]:
trips_df['member_casual'].value_counts()

In [ ]:
trips_df['rideable_type'].value_counts()

In [ ]:
# Initialize base cost
# Start with 0 cost
trips_df['trip_cost'] = 0.0

# Define fixed costs
trips_df.loc[trips_df['member_casual'] == 'member', 'trip_cost'] = 3.95
trips_df.loc[trips_df['member_casual'] == 'casual', 'trip_cost'] = 1.00

# Add extra cost for duration
# for members :
# Create condition for member rides longer than 45 mins
cond_member_extra = (trips_df['member_casual'] == 'member') & (trips_df['trip_duration_minutes'] > 45)

# Electric bike extra for members
trips_df.loc[cond_member_extra & (trips_df['rideable_type'] == 'electric_bike'), 'trip_cost'] += \
    (trips_df['trip_duration_minutes'] - 45) * 0.10

# Classic bike extra for members
trips_df.loc[cond_member_extra & (trips_df['rideable_type'] == 'classic_bike'), 'trip_cost'] += \
    (trips_df['trip_duration_minutes'] - 45) * 0.05
# Electric bike for casuals
cond_casual_electric = (trips_df['member_casual'] == 'casual') & (trips_df['rideable_type'] == 'electric_bike')
trips_df.loc[cond_casual_electric, 'trip_cost'] += trips_df['trip_duration_minutes'] * 0.15

# Classic bike for casuals
cond_casual_classic = (trips_df['member_casual'] == 'casual') & (trips_df['rideable_type'] == 'classic_bike')
trips_df.loc[cond_casual_classic, 'trip_cost'] += trips_df['trip_duration_minutes'] * 0.05
# Add Central Business District (CBD) fee
# Preparaing your geometry points
# Create GeoDataFrame of start points
trips_df['start_point'] = trips_df.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis=1)
trips_df['end_point'] = trips_df.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis=1)
# #  Load CBD Polygon
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=4326)  # Ensures it's in WGS 84


# Convert to GeoDataFrames with correct CRS
start_gdf = gpd.GeoDataFrame(trips_df, geometry='start_point', crs='EPSG:4326').to_crs('EPSG:6933')
end_gdf = gpd.GeoDataFrame(trips_df, geometry='end_point', crs='EPSG:4326').to_crs('EPSG:6933')

# Load CBD polygon and project to EPSG:6933
# CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
# CBD = CBD.to_crs(epsg=6933)
# cbd_polygon = CBD.geometry.unary_union  # Get full boundary

In [ ]:
# Load CBD polygon and project to EPSG:6933
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=6933)
cbd_polygon = CBD.geometry.unary_union  # Get full boundary
CBD

In [ ]:
# Check spatial containment in EPSG:6933
trips_df['start_in_cbd'] = start_gdf['start_point'].apply(lambda point: point.within(cbd_polygon))
trips_df['end_in_cbd'] = end_gdf['end_point'].apply(lambda point: point.within(cbd_polygon))

# Final condition and cost update
trips_df['in_cbd'] = trips_df['start_in_cbd'] | trips_df['end_in_cbd']
trips_df.loc[trips_df['in_cbd'], 'trip_cost'] += 0.5
trips_df['trip_cost'].head()

In [ ]:
trips_df['trip_cost'].describe()

*we can see a clear issue in the data ,  and super high values (4.3 mil in the max ) and std is very high (4837.62) , so we must identify this outliers and deal with them*

In [ ]:
# High-cost trips
high_cost = trips_df[trips_df['trip_cost'] > 1000].copy()
print(high_cost[['ride_id', 'trip_duration_minutes', 'rideable_type', 'member_casual', 'trip_cost']])

# Negative-cost trips
neg_cost = trips_df[trips_df['trip_cost'] < 0].copy()
# try to only print the len
print(neg_cost[['ride_id', 'trip_duration_minutes', 'rideable_type', 'member_casual', 'trip_cost']])


In [ ]:
# Total rows
total_rows = len(trips_df)
# Define thresholds
high_cost_threshold = 10000
negative_cost_threshold = 0

# Find outliers
high_cost_outliers = trips_df[trips_df['trip_cost'] > high_cost_threshold]
negative_cost_outliers = trips_df[trips_df['trip_cost'] < negative_cost_threshold]

# Count
num_high_cost = len(high_cost_outliers)
num_negative_cost = len(negative_cost_outliers)
total_outliers = num_high_cost + num_negative_cost

# Percentages
percent_high_cost = (num_high_cost / total_rows) * 100
percent_negative_cost = (num_negative_cost / total_rows) * 100
percent_total_outliers = (total_outliers / total_rows) * 100

print(f"High cost outliers: {num_high_cost} ({percent_high_cost:.2f}%)")
print(f"Negative cost outliers: {num_negative_cost} ({percent_negative_cost:.2f}%)")
print(f"Total outliers: {total_outliers} ({percent_total_outliers:.2f}%)")


since they make a very small amount of the data (0.0%) they can be classifed as false data and we can drop them

In [ ]:
# Drop outliers by reassigning the filtered DataFrame back to df
trips_df = trips_df[(trips_df['trip_cost'] <= high_cost_threshold) & (trips_df['trip_cost'] >= negative_cost_threshold)]
trips_df['trip_cost'].describe()


---
B4
---


In [ ]:
stations_df['CAPACITY'].describe()

In [ ]:
# Basic histogram using Plotly
fig = px.histogram(stations_df, x='CAPACITY', nbins=30, title='Distribution of Station Capacity')
fig.update_layout(xaxis_title='Capacity', yaxis_title='Count', bargap=0.1)
fig.show(config={'staticPlot': True})


*Choosing the right threshold*

In [ ]:
# Drop NaNs
capacity_data = stations_df['CAPACITY'].dropna()
# Histogram
hist_data = go.Histogram(x=capacity_data, nbinsx=30, name='Histogram', opacity=0.6)
# Density Curve
kde = gaussian_kde(capacity_data)
x_vals = np.linspace(capacity_data.min(), capacity_data.max(), 1000)
kde_data = go.Scatter(x=x_vals, y=kde(x_vals) * len(capacity_data) * (x_vals[1] - x_vals[0]),
                      mode='lines', name='KDE Curve')

# Plot both
fig = go.Figure(data=[hist_data, kde_data])
fig.update_layout(title='Capacity Distribution with KDE',
                  xaxis_title='Capacity', yaxis_title='Count')
# Example thresholds
low_thresh = stations_df['CAPACITY'].quantile(0.30)
high_thresh = stations_df['CAPACITY'].quantile(0.66)
print(low_thresh,high_thresh)
fig.add_vline(x=low_thresh, line_dash="dash", line_color="green", annotation_text="Small/Average")
fig.add_vline(x=high_thresh, line_dash="dash", line_color="red", annotation_text="Average/Large")

fig.show(config={'staticPlot': True})


method 1 : using quantiles

In [ ]:
# Calculate the thresholds
low_thresh = stations_df['CAPACITY'].quantile(0.33)
high_thresh = stations_df['CAPACITY'].quantile(0.66)

def classify_capacity(cap):
    if cap <= low_thresh:
        return 'Small'
    elif cap <= high_thresh:
        return 'Average'
    else:
        return 'Large'

stations_df['STATION_SIZE'] = stations_df['CAPACITY'].apply(classify_capacity)
stations_df['STATION_SIZE'].value_counts()



method 2 : based on domain knowledge


In [ ]:
def classify_capacity(cap):
    if cap <= 15:
        return 'Small'
    elif cap <= 25:
        return 'Average'
    else:
        return 'Large'

stations_df['STATION_SIZE'] = stations_df['CAPACITY'].apply(classify_capacity)
stations_df['STATION_SIZE'].value_counts()
print(stations_df['STATION_SIZE'].value_counts())


combine with trips df

try1 : using the station_id

In [ ]:
# Step 2: Create a simplified DataFrame for merging
station_size_map = stations_df[['STATION_ID', 'STATION_SIZE']].copy()
# Step 3: Merge for start_station_size
trips_df = trips_df.merge(
    station_size_map.rename(columns={
        'STATION_ID': 'start_station_id',
        'STATION_SIZE': 'start_station_size'
    }),
    on='start_station_id',
    how='left'
)

# Step 4: Merge for end_station_size
trips_df = trips_df.merge(
    station_size_map.rename(columns={
        'STATION_ID': 'end_station_id',
        'STATION_SIZE': 'end_station_size'
    }),
    on='end_station_id',
    how='left'
)


In [ ]:
print("Missing start_station_size:", trips_df['start_station_size'].isna().sum())
print("Missing end_station_size:", trips_df['end_station_size'].isna().sum())


In [ ]:
# What kind of start_station_id had no match?
print(trips_df[trips_df['start_station_size'].isna()][['start_station_id']].drop_duplicates().head(10))

# Same for end_station
print(trips_df[trips_df['end_station_size'].isna()][['end_station_id']].drop_duplicates().head(10))


try2 : with names

In [ ]:
stations_df['NAME'] = stations_df['NAME'].str.strip().str.lower()
trips_df['start_station_name'] = trips_df['start_station_name'].str.strip().str.lower()
trips_df['end_station_name'] = trips_df['end_station_name'].str.strip().str.lower()

# Map start station size using name
station_size_name_map = stations_df[['NAME', 'STATION_SIZE']]

trips_df = trips_df.merge(
    station_size_name_map.rename(columns={'NAME': 'start_station_name', 'STATION_SIZE': 'start_station_size_name'}),
    on='start_station_name',
    how='left'
)

# Same for end station
trips_df = trips_df.merge(
    station_size_name_map.rename(columns={'NAME': 'end_station_name', 'STATION_SIZE': 'end_station_size_name'}),
    on='end_station_name',
    how='left'
)


In [ ]:
trips_df['end_station_size'].value_counts()

In [ ]:
trips_df['start_station_size'] = trips_df['start_station_size_name']
trips_df['end_station_size'] = trips_df['end_station_size_name']

# Then drop the temp columns
trips_df.drop(columns=['start_station_size_name', 'end_station_size_name'], inplace=True)


In [ ]:
print(trips_df[['start_station_size', 'end_station_size']].isna().sum())


using names was better but we still have some null values

In [ ]:

fig = px.histogram(stations_df, x='CAPACITY', nbins=30, title='Station Capacity Distribution')
fig.add_vline(x=15, line_dash="dash", line_color="green", annotation_text="Small/Average")
fig.add_vline(x=25, line_dash="dash", line_color="red", annotation_text="Average/Large")

fig.show(config={'staticPlot': True})


---
B5
---



In [ ]:
Shuttle_Bus_Stops.isna().sum()


In [ ]:
Metro_Bus_Stops['BSTP_LAT'].isna().sum()


Approaches


---


| Approach                    | Time Complexity | Vectorized | Fast    |
| --------------------------- | --------------- | ---------- | ------- |
| Brute Force (Your original) | O(N × M)        | ❌ No       | 🐌 Slow |
| BallTree (New)              | O(N log M)      | ✅ Yes      | ⚡ Fast  |


Project all your coordinates to EPSG:6933


In [ ]:
# Create start and end point geometries
trips_df['start_point'] = trips_df.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis=1)
trips_df['end_point'] = trips_df.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis=1)

# Create GeoDataFrames
gdf_start = gpd.GeoDataFrame(trips_df, geometry='start_point', crs='EPSG:4326').to_crs(epsg=6933)
gdf_end = gpd.GeoDataFrame(trips_df, geometry='end_point', crs='EPSG:4326').to_crs(epsg=6933)

# Add x/y columns
trips_df['start_x'] = gdf_start.geometry.x
trips_df['start_y'] = gdf_start.geometry.y
trips_df['end_x'] = gdf_end.geometry.x
trips_df['end_y'] = gdf_end.geometry.y


# projecting   metro and shuttle station coordinates:

# Convert station lat/lng to projected coordinates
def project_coords(coords_list):
    gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coords_list], crs='EPSG:4326')
    gdf = gdf.to_crs(epsg=6933)
    return np.array([(geom.x, geom.y) for geom in gdf.geometry])
# coords
# Metro stop coordinates
metro_coords = Metro_Bus_Stops[['BSTP_LAT', 'BSTP_LON']].dropna().values

# Shuttle stop coordinates
shuttle_coords = Shuttle_Bus_Stops[['LATITUDE', 'LONGITUDE']].dropna().values

metro_coords_projected = project_coords(metro_coords)
shuttle_coords_projected = project_coords(shuttle_coords)


In [ ]:

def euclidean_tree_batch(source_df, stop_coords, x_col, y_col, batch_size=10000):
    tree = BallTree(stop_coords, metric='euclidean')

    distances = []
    n = len(source_df)
    tqdm.pandas(desc=f"Computing distances for {x_col}")

    for i in tqdm(range(0, n, batch_size), desc="Batch processing", unit="batch"):
        batch = source_df.iloc[i:i+batch_size]
        batch_points = batch[[x_col, y_col]].values

        dists, _ = tree.query(batch_points, k=1)
        distances.extend(dists.flatten().tolist())

    return distances


In [ ]:
# Start → Metro
trips_df['start_nearest_metro_distance'] = euclidean_tree_batch(
    trips_df, metro_coords_projected, 'start_x', 'start_y'
)

# End → Metro
trips_df['end_nearest_metro_distance'] = euclidean_tree_batch(
    trips_df, metro_coords_projected, 'end_x', 'end_y'
)

# Start → Shuttle
trips_df['start_nearest_shuttle_distance'] = euclidean_tree_batch(
    trips_df, shuttle_coords_projected, 'start_x', 'start_y'
)

# End → Shuttle
trips_df['end_nearest_shuttle_distance'] = euclidean_tree_batch(
    trips_df, shuttle_coords_projected, 'end_x', 'end_y'
)

# converting to from meters to km (our choice)
trips_df['start_nearest_metro_distance'] = trips_df['start_nearest_metro_distance'] / 1000
trips_df['end_nearest_metro_distance'] = trips_df['end_nearest_metro_distance'] / 1000
trips_df['start_nearest_shuttle_distance'] = trips_df['start_nearest_shuttle_distance'] / 1000
trips_df['end_nearest_shuttle_distance'] = trips_df['end_nearest_shuttle_distance'] / 1000


In [ ]:
trips_df[
    ['start_nearest_metro_distance',
     'end_nearest_metro_distance',
     'start_nearest_shuttle_distance',
     'end_nearest_shuttle_distance']
].describe()


In [ ]:
sampled_df = trips_df.sample(n=20000, random_state=50)

cols = ['start_nearest_metro_distance', 'end_nearest_metro_distance',
        'start_nearest_shuttle_distance', 'end_nearest_shuttle_distance']
for col in cols:
    fig = go.Figure(
        data=[go.Histogram(
            x=sampled_df[col],
            nbinsx=100,
            marker=dict(color='skyblue'),
            opacity=0.75
        )]
    )
    fig.update_layout(
        title=col,
        xaxis_title=col,
        yaxis_title='Count (Log Scale)',
        yaxis_type='log',
        bargap=0.1,
        width=800,
        height=400
    )
    fig.show(config={'staticPlot':True})

In [ ]:
import folium
import pandas as pd # Assuming trips_df is a pandas DataFrame

# Re-define thresholds for clarity
start_nearest_metro_distance_thr = 1 # meters
end_nearest_metro_distance_thr = 1   # meters
start_nearest_shuttle_distance_thr = 9 # meters
end_nearest_shuttle_distance_thr = 9 # meters

# --- Step 1: Identify "far from" trips based on current thresholds ---
# Using the corrected end_nearest_shuttle_distance_thr for the end shuttle distance
far_metro_start_trips = trips_df[trips_df['start_nearest_metro_distance'] > start_nearest_metro_distance_thr]
far_metro_end_trips = trips_df[trips_df['end_nearest_metro_distance'] > end_nearest_metro_distance_thr]
far_shuttle_start_trips = trips_df[trips_df['start_nearest_shuttle_distance'] > start_nearest_shuttle_distance_thr]
far_shuttle_end_trips = trips_df[trips_df['end_nearest_shuttle_distance'] > end_nearest_shuttle_distance_thr]

# Combine all "far from transit" trips for a general map (for demonstration)
# Using a logical OR to get any trip that is far from ANY of these points
far_from_transit_trips = trips_df[
    (trips_df['start_nearest_metro_distance'] > start_nearest_metro_distance_thr) |
    (trips_df['end_nearest_metro_distance'] > end_nearest_metro_distance_thr) |
    (trips_df['start_nearest_shuttle_distance'] > start_nearest_shuttle_distance_thr) |
    (trips_df['end_nearest_shuttle_distance'] > end_nearest_shuttle_distance_thr)
].copy() # Use .copy() to avoid SettingWithCopyWarning

print(f"Number of trips far from metro (start): {len(far_metro_start_trips)}")
print(f"Number of trips far from metro (end): {len(far_metro_end_trips)}")
print(f"Number of trips far from shuttle (start): {len(far_shuttle_start_trips)}")
print(f"Number of trips far from shuttle (end): {len(far_shuttle_end_trips)}")
print(f"Total unique trips identified as 'far from transit': {len(far_from_transit_trips)}")


# --- Step 2: Create a Folium Map ---

# Get the approximate center of Washington D.C. for the map's initial view
# You can use the mean of your start/end lat/lngs or a known DC coordinate
dc_center_lat = trips_df['start_lat'].mean() # Or a more precise known center for DC
dc_center_lng = trips_df['start_lng'].mean() # Or a more precise known center for DC

# Create a base map
m = folium.Map(location=[dc_center_lat, dc_center_lng], zoom_start=12)

# Add markers for start points of trips identified as "far from transit"
# Due to the large number of potential points (40k), plotting individual markers for all
# might be slow or make the map unreadable.
# We'll plot a sample or use a MarkerCluster for better performance.
# Let's start by plotting a *sample* of these points if far_from_transit_trips is very large,
# or use MarkerCluster. For a first look, a small sample is good.

# If you have too many points, consider sampling for initial visualization
# Or, even better for density visualization, use MarkerCluster or HeatMap (if allowed for density, check project rules)
# Since you're using Plotly for charts and Folium for maps, heatmap should be fine.

# Let's just add a few to see the logic work, or use MarkerCluster for all:

# OPTION A: Plotting a limited sample (good for a quick check if map gets cluttered)
# sample_size = 1000 # Adjust as needed
# if len(far_from_transit_trips) > sample_size:
#     sample_to_plot = far_from_transit_trips.sample(sample_size, random_state=42)
# else:
#     sample_to_plot = far_from_transit_trips

# for idx, row in sample_to_plot.iterrows():
#     folium.CircleMarker(
#         location=[row['start_lat'], row['start_lng']],
#         radius=2, # Small radius
#         color='red',
#         fill=True,
#         fill_color='red',
#         fill_opacity=0.6,
#         tooltip=f"Start: {row['start_station_name']} (Far from Transit)"
#     ).add_to(m)

# OPTION B: Using MarkerCluster for better visualization of many points
from folium.plugins import MarkerCluster

marker_cluster_start = MarkerCluster().add_to(m)
marker_cluster_end = MarkerCluster().add_to(m)

# Add start points
for idx, row in far_from_transit_trips.iterrows():
    if pd.notnull(row['start_lat']) and pd.notnull(row['start_lng']):
        folium.CircleMarker(
            location=[row['start_lat'], row['start_lng']],
            radius=2,
            color='red', # Color for start points
            fill=True,
            fill_color='red',
            fill_opacity=0.6,
            tooltip=f"Start: {row['start_station_name']} (Far from Transit)"
        ).add_to(marker_cluster_start)

# Add end points (optional, you might want separate layers or colors if combining)
# For now, let's just show start points to avoid overwhelming the map.
# If you want to see end points, you could use a different color or a separate MarkerCluster
# for idx, row in far_from_transit_trips.iterrows():
#     if pd.notnull(row['end_lat']) and pd.notnull(row['end_lng']):
#         folium.CircleMarker(
#             location=[row['end_lat'], row['end_lng']],
#             radius=2,
#             color='blue', # Color for end points
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6,
#             tooltip=f"End: {row['end_station_name']} (Far from Transit)"
#         ).add_to(marker_cluster_end)


# Save the map to an HTML file or display in a Jupyter Notebook
# m.save("far_from_transit_trips_map.html")
# In a Jupyter/IPython environment, you can also just display `m` directly
m

In [ ]:
# --- Define your new thresholds based on your histogram observations ---
# Example new thresholds (YOU WILL REPLACE THESE WITH YOUR OWN INSIGHTS)
new_metro_start_thr = 1  # meters (e.g., if you see a clear drop after 1.2km)
new_metro_end_thr = 1    # meters
new_shuttle_start_thr = 9 # meters (e.g., if you see a drop after 20km)
new_shuttle_end_thr = 9 # meters

# --- Create the new boolean features ---
trips_df['is_far_from_metro_start'] = trips_df['start_nearest_metro_distance'] > new_metro_start_thr
trips_df['is_far_from_metro_end'] = trips_df['end_nearest_metro_distance'] > new_metro_end_thr
trips_df['is_far_from_shuttle_start'] = trips_df['start_nearest_shuttle_distance'] > new_shuttle_start_thr
trips_df['is_far_from_shuttle_end'] = trips_df['end_nearest_shuttle_distance'] > new_shuttle_end_thr

# You can also create a combined flag for any "far from transit"
trips_df['is_far_from_any_transit'] = (
    trips_df['is_far_from_metro_start'] |
    trips_df['is_far_from_metro_end'] |
    trips_df['is_far_from_shuttle_start'] |
    trips_df['is_far_from_shuttle_end']
)

# Verify the counts of the new features
print("\nCounts for new 'far from' features:")
print(trips_df[['is_far_from_metro_start', 'is_far_from_metro_end',
                'is_far_from_shuttle_start', 'is_far_from_shuttle_end',
                'is_far_from_any_transit']].sum())

# Display the first few rows with the new columns to confirm
print("\nTrips DataFrame with new features:")
print(trips_df[['ride_id', 'start_nearest_metro_distance', 'is_far_from_metro_start',
                'start_nearest_shuttle_distance', 'is_far_from_shuttle_start',
                'is_far_from_any_transit']].head())

In [ ]:
trips_df['is_far_from_metro_start'].value_counts()

---
B6
---


In [ ]:
print(trips_df['start_point'].iloc[0], type(trips_df['start_point'].iloc[0]))
print(trips_df['end_point'].iloc[0], type(trips_df['end_point'].iloc[0]))
print(type(cbd_polygon))


In [ ]:
# STEP 0: Make sure the CBD polygon is projected correctly
CBD = CBD.to_crs(epsg=6933)
cbd_polygon = CBD.geometry.iloc[0]  # assuming a single polygon
# STEP 1: Create a GeoDataFrame from the trip points (start and end)
# start_gdf = gpd.GeoDataFrame(trips_df, geometry=trips_df['start_point'], crs="EPSG:4326")
# end_gdf   = gpd.GeoDataFrame(trips_df, geometry=trips_df['end_point'], crs="EPSG:4326")

# Rebuild the point geometries from lat/lng in EPSG:4326
start_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['start_lng'], trips_df['start_lat']),
    crs="EPSG:4326"
)

end_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs="EPSG:4326"
)


# Project everything to EPSG:6933
CBD = CBD.to_crs(epsg=6933)
start_gdf = start_gdf.to_crs(epsg=6933)
end_gdf = end_gdf.to_crs(epsg=6933)

# CBD polygon (in same projection)
cbd_polygon = CBD.geometry.unary_union
# Check containment
trips_df['start_in_cbd'] = start_gdf['geometry'].apply(lambda pt: cbd_polygon.contains(pt))
trips_df['end_in_cbd']   = end_gdf['geometry'].apply(lambda pt: cbd_polygon.contains(pt))

# Final result
trips_df['in_cbd'] = trips_df['start_in_cbd'] | trips_df['end_in_cbd']
trips_df['in_cbd'].value_counts()

---
B7
---


In [ ]:
# --- Step 1: Compute the CBD centroid (already in EPSG:6933)
cbd_centroid = cbd_polygon.centroid  # geometry in meters (EPSG:6933)

# --- Step 2: Recreate end point GeoDataFrame and project to EPSG:6933
end_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs="EPSG:4326"
).to_crs(epsg=6933)

# --- Step 3: Compute Euclidean distance in meters
trips_df['distance_to_cbd_m'] = end_gdf.geometry.distance(cbd_centroid)

# --- Step 4: Set distance to None where start AND end are in the CBD
mask = trips_df['start_in_cbd'] & trips_df['end_in_cbd']
trips_df.loc[mask, 'distance_to_cbd_m'] = None

# --- Step 5: Inspect result
trips_df['distance_to_cbd_m'].describe()




**Threasholding strategies**





kinda of an elbow method

In [ ]:
sampled_df = trips_df.sample(n=20000, random_state=50)

# Extract the data
data = sampled_df['distance_to_cbd_m'].dropna()

# Create histogram trace
hist = go.Histogram(
    x=data,
    nbinsx=100,
    name='Histogram',
    marker_color='lightblue',
    opacity=0.75
)

# Create KDE line (manual since Plotly doesn’t support KDE directly)
kde = gaussian_kde(data)
x_vals = np.linspace(data.min(), data.max(), 1000)
kde_vals = kde(x_vals) * len(data) * (x_vals[1] - x_vals[0])  # scale to match histogram

kde_trace = go.Scatter(
    x=x_vals,
    y=kde_vals,
    mode='lines',
    name='KDE',
    line=dict(color='darkblue')
)

# Vertical reference lines
vline1 = go.Scatter(
    x=[2000, 2000],
    y=[0, max(kde_vals)],
    mode='lines',
    name='2km Threshold',
    line=dict(color='red', dash='dash')
)

vline2 = go.Scatter(
    x=[2764, 2764],
    y=[0, max(kde_vals)],
    mode='lines',
    name='Median',
    line=dict(color='green', dash='dash')
)

# Create the figure
fig = go.Figure(data=[hist, kde_trace, vline1, vline2])

# Update layout
fig.update_layout(
    title='Distance to CBD at End of Trip',
    xaxis_title='distance_to_cbd_m',
    yaxis_title='Count',
    width=800,
    height=500,
    legend=dict(x=0.7, y=0.95)
)

fig.show( config={'staticPlot':True})


In [ ]:
"""
i will choose the median beacause looking at the histogram we can see the counts drops
"""
threshold = 2764
# Apply binary classification
trips_df['close_to_cbd'] = trips_df['distance_to_cbd_m'].apply(
    lambda d: None if pd.isna(d) else d <= threshold
)
trips_df['close_to_cbd'].value_counts()

In [ ]:
print(trips_df['close_to_cbd'].isna().sum())

---
B8
---




In [ ]:
"""
Washington, D.C. is roughly:

~16 km (north-south)

~13 km (east-west)

So, a geohash precision of 5–8 is appropriate.
"""
def encode_geohashes(df, lat_col, lon_col, precisions):
    for p in precisions:
        col_name = f'geohash_p{p}'
        df[col_name] = df.apply(lambda row: geohash2.encode(row[lat_col], row[lon_col], p), axis=1)
    return df

# Try precisions from 5 to 8
precisions_to_test = [5, 6, 7, 8]
trips_df = encode_geohashes(trips_df, 'start_lat', 'start_lng', precisions_to_test)
for p in precisions_to_test:
    print(f"Precision {p}: {trips_df[f'geohash_p{p}'].nunique()} unique regions")
"""
If the number is too small → you're over-aggregating.

If it's too big (e.g. thousands) → too fine → hard to summarize meaningfully.
"""

for p in precisions_to_test:
    counts = trips_df[f'geohash_p{p}'].value_counts()
    print(f"Precision {p} → median trips per geohash: {counts.median()}")
"""
This tells us how balanced the spatial bins are.

we ideally want 50–500 trips per cell.
"""

| Precision | Median Trips per Geohash | Interpretation                                                     |
| --------- | ------------------------ | ------------------------------------------------------------------ |
| **5**     | 1761                     | ⚠️ Too coarse — merges many neighborhoods into one.                |
| **6**     | 196                      | ✅ Good balance — each area has enough trips for reliable analysis. |
| **7**     | 7                        | ⚠️ Very fine — may be too sparse for most practical summaries.     |
| **8**     | 2                        | 🚫 Too sparse — most areas will be noise or empty.                 |


In [ ]:
# we will choose 6t
trips_df['geohash_sector'] = trips_df['geohash_p6']


---

B9
---


In [ ]:
# Group by Sector and Date
# Assume you have a 'date' column (convert if needed)
trips_df['date'] = pd.to_datetime(trips_df['date'])

# Count trips per day per sector
daily_counts = trips_df.groupby(['geohash_p6', 'date']).size().reset_index(name='trip_count')

# Now compute average daily trips per geohash sector
avg_daily_trips = daily_counts.groupby('geohash_p6')['trip_count'].mean().reset_index()
avg_daily_trips.rename(columns={'trip_count': 'avg_daily_trips'}, inplace=True)


Choose Segmentation Method (for Red / Yellow / Gray)


| Method                         | Description                          | Pros             | Use Case             |
| ------------------------------ | ------------------------------------ | ---------------- | -------------------- |
| **Quantiles** (e.g., tertiles) | Divide into 3 equal-sized groups     | Simple, fair     | Balanced datasets    |
| **Natural Breaks (Jenks)**     | Optimize separation between clusters | Data-aware       | Uneven distributions |
| **KMeans Clustering (k=3)**    | Machine learning-based segmentation  | Optimal grouping | Large datasets       |


In [ ]:
# quantiles :
# Assign labels based on quantiles
quantiles = avg_daily_trips['avg_daily_trips'].quantile([1/3, 2/3])
low_thresh = quantiles.iloc[0]
high_thresh = quantiles.iloc[1]

def classify_volume(val):
    if val < low_thresh:
        return 'gray'   # Low volume
    elif val < high_thresh:
        return 'yellow' # Medium volume
    else:
        return 'red'    # High volume

avg_daily_trips['volume_segment'] = avg_daily_trips['avg_daily_trips'].apply(classify_volume)


In [ ]:

# Extract the data
data = avg_daily_trips['avg_daily_trips'].dropna()

# Histogram trace
hist = go.Histogram(
    x=data,
    nbinsx=30,
    marker_color='lightblue',
    opacity=0.75,
    name='Avg Daily Trips'
)

# Vertical threshold lines
vline_low = go.Scatter(
    x=[low_thresh, low_thresh],
    y=[0, data.value_counts().max()],
    mode='lines',
    name='Low Threshold',
    line=dict(color='gray', dash='dash')
)

vline_high = go.Scatter(
    x=[high_thresh, high_thresh],
    y=[0, data.value_counts().max()],
    mode='lines',
    name='High Threshold',
    line=dict(color='orange', dash='dash')
)

# Combine into figure
fig = go.Figure(data=[hist, vline_low, vline_high])

# Update layout
fig.update_layout(
    title='Distribution of Avg Daily Trips per Geohash Sector',
    xaxis_title='Avg Daily Trips',
    yaxis_title='Count',
    width=800,
    height=500,
    bargap=0.1
)

fig.show(config={'staticPlot':True})


In [ ]:
X = avg_daily_trips[['avg_daily_trips']].values

kmeans = KMeans(n_clusters=3, random_state=42  , n_init=10).fit(X)
avg_daily_trips['kmeans_label'] = kmeans.labels_

# Map to red/yellow/gray using sorted cluster means
label_map = dict(zip(
    np.argsort(kmeans.cluster_centers_.flatten()),
    ['gray', 'yellow', 'red']
))
avg_daily_trips['kmeans_segment'] = avg_daily_trips['kmeans_label'].map(label_map)


In [ ]:
avg_daily_trips.head()

In [ ]:
trips_df['geohash_p6'].nunique()

In [ ]:
# Merge segments into trips_df
trips_df = trips_df.merge(
    avg_daily_trips[['geohash_p6','volume_segment','kmeans_segment']],
    on='geohash_p6',
    how='left'
)


In [ ]:
trips_df['kmeans_segment'].value_counts()


In [ ]:
trips_df['volume_segment'].value_counts()



---

B10
----

In [ ]:
trips_df['conditions'].value_counts()

In [ ]:
def classify_weather(condition):
    condition = condition.lower()  # lowercase for safety
    if 'rain' in condition or 'snow' in condition:
        return 'rainy'
    elif 'overcast' in condition or 'cloudy' in condition:
        return 'cloudy'
    elif 'clear' in condition:
        return 'sunny'
    else:
        return 'unknown'

# Apply binning
trips_df['weather_segment'] = trips_df['conditions'].apply(classify_weather)
trips_df['weather_segment'].value_counts()

---

B11
---

quick inspection of the data dates

In [ ]:
sorted_ended_at_df = trips_df[['ended_at']].sort_values(by='ended_at')
print("--- Sorted 'ended_at' DataFrame (first 5 rows) ---")
print(sorted_ended_at_df.head())
print("\n")

# --- Step 3: Find the earliest and latest dates ---
earliest_date = sorted_ended_at_df['ended_at'].min()
latest_date = sorted_ended_at_df['ended_at'].max()

print(f"The earliest date in 'ended_at' is: {earliest_date}")
print(f"The latest date in 'ended_at' is: {latest_date}")


In [ ]:

# --- Step 2: Sort the DataFrame by 'started_at' ---
sorted_started_at_df = trips_df[['started_at']].sort_values(by='started_at')
print("--- Sorted 'started_at' DataFrame (first 5 rows) ---")
print(sorted_started_at_df.head())
print("\n")

# --- Step 3: Find the earliest and latest dates using 'started_at' ---
earliest_date_started = sorted_started_at_df['started_at'].min()
latest_date_started = sorted_started_at_df['started_at'].max()

print(f"The earliest date in 'started_at' is: {earliest_date_started}")
print(f"The latest date in 'started_at' is: {latest_date_started}")


In [ ]:
# Make sure 'ended_at' is datetime
# trips_df['ended_at'] = pd.to_datetime(trips_df['ended_at'])
trips_df['ended_at'] = pd.to_datetime(trips_df['ended_at'], format='mixed', errors='coerce')


# Extract just the date (without time)
trips_df['end_date'] = trips_df['ended_at'].dt.date
daily_income_weather = trips_df.groupby(['end_date', 'weather_segment'])['trip_cost'].sum().reset_index()



In [ ]:
# convert
# Make sure end_date is datetime
daily_income_weather['end_date'] = pd.to_datetime(daily_income_weather['end_date'])

fig_long = px.line(
    daily_income_weather,
    x='end_date',
    y='trip_cost',
    color='weather_segment',
    title='Daily Total Trip Cost by Weather Condition (Long Format)',
    labels={'end_date': 'Date', 'trip_cost': 'Total Income', 'weather_segment': 'Weather'}
)

fig_long.update_layout(xaxis_title='Date', yaxis_title='Trip Cost', hovermode='x unified')
fig_long.show(config={'staticPlot':True})


In [ ]:
# Pivot to wide format
wide_df = daily_income_weather.pivot(index='end_date', columns='weather_segment', values='trip_cost').fillna(0)
wide_df = wide_df.sort_index()

# Build traces
fig_wide = go.Figure()

for condition in wide_df.columns:
    fig_wide.add_trace(go.Scatter(
        x=wide_df.index,
        y=wide_df[condition],
        mode='lines',
        name=condition
    ))

fig_wide.update_layout(
    title='Daily Total Trip Cost by Weather Condition (Wide Format)',
    xaxis_title='Date',
    yaxis_title='Trip Cost',
    hovermode='x unified',
    template='plotly_white',
    legend_title='Weather'
)

fig_wide.show(config={'staticPlot':True})


Which one is better for our problem  ?<br>
the Long Format is the most suitable and effective,This is because it allows for direct visual comparison of revenue trends across different weather types over time on a single graph, making it easier to spot patterns and seasonal impacts. The long format is also considered more intuitive for time-series visualization. Conversely, the "wide format" is deemed less clear due to potential visual clutter when many categories are present.


---
B12
---

Feature 1 : rush_hour
<br> Indicates if the ride occurred during typical commuting hours (7–10 AM or 4–7 PM).

In [ ]:

trips_df['start_time'] = pd.to_datetime(trips_df['start_time'], errors='coerce')

trips_df['rush_hour'] = (
    trips_df['start_time'].dt.hour.between(7, 10) |
    trips_df['start_time'].dt.hour.between(16, 19)
).astype(int)
trips_df['rush_hour'].value_counts()


Feature 2 : hour_segment <br>
Categorize ride start times into broader buckets.

In [ ]:
def get_hour_segment(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Midday'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

trips_df['hour_segment'] = trips_df['start_time'].dt.hour.apply(get_hour_segment)
trips_df['hour_segment'].value_counts()


Feature 3 : is_weekend<br>
Helps spot usage patterns on weekends vs weekdays.

In [ ]:
trips_df['is_weekend'] = trips_df['start_time'].dt.dayofweek >= 5
trips_df['is_weekend'] = trips_df['is_weekend'].astype(int)
trips_df['is_weekend'].value_counts()



---


#**EDA**


---





---
Sampling the data
---



In [ ]:

# Sampled data stats
sampled_df = trips_df.sample(n=20000, random_state=50)





---

# A )


---



## Task 1: Top 5 Starting Stations Analysis

### Objective
Identify the top 5 stations with the highest number of trip departures (starting stations) and create a bar chart showing statistical information for these top 5 stations.

### Requirements
- Identify the top 5 stations with the highest number of trip departures (starting stations)
- Create a bar chart showing statistical information for these top 5 stations
- Display the count of trips starting from each station


In [ ]:

# Identify the top 5 starting stations
start_station_counts = sampled_df['start_station_name'].value_counts()
top_5_start_stations = start_station_counts.head(5)

print("Top 5 Starting Stations:")
print(top_5_start_stations)

# Create a bar chart for the top 5 starting stations
fig = px.bar(
    top_5_start_stations,
    x=top_5_start_stations.index,
    y=top_5_start_stations.values,
    title='Top 5 Starting Stations by Trip Count',
    labels={'x': 'Station Name', 'y': 'Number of Trips'},
    color=top_5_start_stations.values,  # Color bars by count
    color_continuous_scale=px.colors.sequential.Viridis # Optional: choose a color scale
)

# Update layout for better readability
fig.update_layout(
    xaxis={'categoryorder':'total descending'}, # Ensure bars are ordered by count
    xaxis_title='Station Name',
    yaxis_title='Number of Trips',
    hovermode='x unified'
)

fig.show(config={'staticPlot': True})

### Top 5 Starting Stations by Trip Count

**Key Insight:**

* The station **"Park Rd & Holmead Pl NW"** is by far the **most popular starting point**, with approximately **300 trips**, significantly ahead of the next stations.

**Detailed Breakdown:**

* The remaining top 4 stations:

  * **"14th & Belmont St NW"**
  * **"18th St & Wyoming Ave NW"**
  * **"Columbus Circle / Union Station"**
  * **"Lamont & Mt Pleasant NW"**

  Each of these falls in the **190–210 trip range**, indicating moderate popularity and relatively close usage levels among them.

**Implication:**

* The high volume at **Park Rd & Holmead Pl NW** could reflect factors such as:

  * Proximity to residential or commercial zones
  * Availability of bike lanes or connectivity
  * Strategic location near transit hubs or universities

This station likely plays a **central role in trip generation**, which may warrant priority in terms of maintenance, expansions, or targeted promotions.

---

## Task 2: Trip Distribution by Bike Type and Membership

### Objective
Calculate the distribution of trips by bike type (classic vs electric) and membership type (member vs casual), then create a single bar chart showing these distributions.

### Requirements
- Calculate the distribution of trips by:
  - Bike type (classic vs electric)
  - Membership type (member vs casual)
- Create one bar chart showing these distributions

In [ ]:

# Calculate the distribution
distribution = sampled_df.groupby(['rideable_type', 'member_casual']).size().reset_index(name='count')

# Create the bar chart
fig = px.bar(
    distribution,
    x='rideable_type',
    y='count',
    color='member_casual',
    barmode='group',
    title='Trip Distribution by Bike Type and Membership',
    labels={'rideable_type': 'Bike Type', 'count': 'Number of Trips', 'member_casual': 'Membership Type'}
)

fig.update_layout(xaxis_title='Bike Type', yaxis_title='Number of Trips')
fig.show(config={'staticPlot': True})


### Trip Distribution by Bike Type and Membership

**Key Insights:**

#### 1. **Electric bikes are the dominant preference:**

* **Members** show a strong preference for **electric bikes** (≈7,500 trips), much higher than classic bikes.
* Even **casual users** prefer electric bikes (≈3,700 trips) over classic ones.

#### 2. **Membership significantly boosts usage:**

* Both bike types have **higher trip counts among members** than casual users.
* Members took nearly **double the trips** compared to casuals, suggesting:

  * Regular commuting behavior
  * Higher cost-effectiveness for frequent riders
  * Greater engagement with the system

#### 3. **Classic bikes are less used overall:**

* Classic bikes saw fewer total trips for both user types, particularly from casual riders, possibly due to:

  * Higher effort required
  * Less appeal in convenience and speed

**Implication:**

* The system should consider **prioritizing electric bike availability**, especially in high-traffic stations and for members.
* **Member-focused incentives** or **electric bike maintenance** should be top priorities to support usage trends.

---

## Task 3: Sunburst Chart for Top 5 Starting Stations

### Objective
Create a sunburst chart showing the breakdown of trips for the top 5 starting stations with the hierarchy: Station → Bike Type → Membership Type. This will show how trips are distributed across bike types and membership types for each top station.

### Requirements
- Create a sunburst chart showing the breakdown of trips for the top 5 starting stations
- Show the hierarchy: Station → Bike Type → Membership Type
- Display how trips are distributed across bike types and membership types for each top station


In [ ]:

# Filter data for only the top 5 starting stations
trips_top5_stations = sampled_df[sampled_df['start_station_name'].isin(top_5_start_stations.index)].copy()

# Group data for the sunburst chart
sunburst_data = trips_top5_stations.groupby(['start_station_name', 'rideable_type', 'member_casual']).size().reset_index(name='count')

# Create the sunburst chart
fig_sunburst = px.sunburst(
    sunburst_data,
    path=['start_station_name', 'rideable_type', 'member_casual'],  # Hierarchy
    values='count',
    title='Trip Breakdown for Top 5 Starting Stations by Bike and Membership Type'
)

# Update layout for better appearance
fig_sunburst.update_layout(
    margin=dict(t=0, l=0, r=0, b=0),
    title_text='Trip Breakdown for Top 5 Starting Stations by Bike and Membership Type',
    title_x=0.5 # Center the title
)

fig_sunburst.show(config={'staticPlot': True})


### Trip Breakdown by Start Station, Bike Type, and Membership

The sunburst chart provides a **multidimensional view** of trip distribution across three variables:

1. **Starting Station**
2. **Bike Type (electric or classic)**
3. **Membership Type (casual or member)**

#### **Key Observations:**

1. **Electric Bikes Dominate at Most Stations**

   * At nearly every station, **electric bikes** occupy the largest segment.
   * Most **member** trips originate using electric bikes, indicating a clear usage preference.

2. **Park Rd & Holmead Pl NW Leads in Total Trips**

   * This station has the **widest outer ring**, especially in electric bike usage by members.
   * Indicates it is a **strategic hub**—possibly due to location, connectivity, or accessibility.

3. **Membership-Driven Usage**

   * Across all stations, **member users consistently take more trips** than casual users.
   * Reflects a higher engagement and potential for loyalty-driven programs.

4. **Small Classic Bike Usage by Casuals**

   * In many stations (e.g., 14th & Belmont St NW), classic bikes used by casuals represent a **very thin slice**.
   * Suggests low attractiveness of classic bikes to non-regular riders.

5. **Columbus Circle / Union Station: Balanced Mix**

   * This station shows a relatively **balanced use** of both bike types across membership types.
   * Indicates it serves diverse rider profiles—possibly due to being a major transit point.
   
---

## Task 4: Station Capacity Analysis

### Objective
Create a histogram showing the distribution of station capacities, then create a bar chart showing trip distribution by station capacity categories (small, medium, large). Stations need to be categorized into capacity groups first.

### Requirements
- Create a histogram showing the distribution of station capacities
- Create a bar chart showing trip distribution by station capacity categories (small, medium, large)
- Categorize stations into capacity groups first


In [ ]:

# Merge station size back to trips_df
# Ensure station_id types match
stations_df['STATION_ID'] = stations_df['STATION_ID'].astype(sampled_df['start_station_id'].dtype)

# Merge station size to trips_df based on start station
trips_with_station_size = sampled_df.merge(
    stations_df[['STATION_ID', 'STATION_SIZE']],
    left_on='start_station_id',
    right_on='STATION_ID',
    how='left'
)

# Rename column for clarity
trips_with_station_size.rename(columns={'STATION_SIZE': 'start_station_size'}, inplace=True)

# Merge station size based on end station as well
trips_with_station_size = trips_with_station_size.merge(
    stations_df[['STATION_ID', 'STATION_SIZE']],
    left_on='end_station_id',
    right_on='STATION_ID',
    how='left',
    suffixes=('', '_end_station') # Add suffix for the end station size column
)
trips_with_station_size.rename(columns={'STATION_SIZE': 'end_station_size'}, inplace=True)

# Drop the redundant STATION_ID columns from the merges
trips_with_station_size.drop(columns=['STATION_ID', 'STATION_ID_end_station'], errors='ignore', inplace=True)


# Count trips by start station capacity category
trip_distribution_by_capacity = trips_with_station_size['start_station_size'].value_counts().reset_index()
trip_distribution_by_capacity.columns = ['Station Capacity Category', 'Number of Trips']

# Define category order
category_order = ['Small', 'Average', 'Large']
trip_distribution_by_capacity['Station Capacity Category'] = pd.Categorical(
    trip_distribution_by_capacity['Station Capacity Category'], categories=category_order, ordered=True
)
trip_distribution_by_capacity = trip_distribution_by_capacity.sort_values('Station Capacity Category')


# Create bar chart
fig = px.bar(
    trip_distribution_by_capacity,
    x='Station Capacity Category',
    y='Number of Trips',
    title='Trip Distribution by Start Station Capacity Category',
    labels={'Station Capacity Category': 'Start Station Size', 'Number of Trips': 'Number of Trips'},
    color='Station Capacity Category',
    category_orders={'Station Capacity Category': category_order} # Enforce the order
)

fig.update_layout(xaxis_title='Start Station Capacity Category', yaxis_title='Number of Trips')
fig.show(config={'staticPlot': True})

### Trip Distribution by Start Station Capacity Category

This bar chart displays the **number of trips** originating from stations grouped by their **capacity size**:

* **Small**
* **Average**
* **Large**

#### **Key Insights:**

1. **Average-Capacity Stations Lead in Usage**

   * With nearly **700,000 trips**, average-size stations dominate trip origination.
   * Indicates a **sweet spot** in terms of infrastructure: large enough to support volume, yet compact enough to be conveniently located.

2. **Small Stations Show Strong Engagement**

   * Surprisingly, **small stations** come second, with over **350,000 trips**.
   * Suggests good utilization of micro-mobility infrastructure, possibly in high-density areas or residential zones.

3. **Underperformance of Large Stations**

   * Large-capacity stations contribute the **least number of trips (\~160,000)**.
   * This could indicate:

     * Poor placement (e.g., less foot traffic)
     * Oversupply of docks
     * Underutilized transit hubs




---

# B)

---





---
Task 1
---



| Method                     | Formula                         | Notes                               |
| -------------------------- | ------------------------------- | ----------------------------------- |
| **Sturges’ Rule**          | `bins = ceil(log2(n) + 1)`      | Good for small to medium-sized data |
| **Freedman–Diaconis Rule** | `bin_width = 2 * IQR / n^(1/3)` | Good for skewed data or outliers    |
| **Square Root Rule**       | `bins = sqrt(n)`                | Simple and often a good baseline    |


In [ ]:
# Use the sampled dataframe to avoid memory issues
durations = sampled_df['trip_duration_minutes']

# Freedman–Diaconis rule for bin width
q25, q75 = np.percentile(durations, [25, 75])
iqr = q75 - q25
n = len(durations)
bin_width = 2 * iqr / (n ** (1/3))
bin_count = int(np.ceil((durations.max() - durations.min()) / bin_width))

print(f"Suggested bin count: {bin_count}")

# Static histogram
fig = go.Figure(
    data=[go.Histogram(
        x=durations,
        nbinsx=bin_count,
        marker_color='blue',
        opacity=1.0
    )]
)

fig.update_layout(
    title="Distribution of Trip Duration (in Minutes)",
    xaxis_title="Trip Duration (minutes)",
    yaxis_title="Frequency",
    bargap=0.05,
    template='simple_white'
)

fig.show(config={'staticPlot': True})


How many trips took more then a day ?

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Choose your cutoff (in minutes)
cutoff = 1440  # Modify as needed

# Use the sampled dataframe to avoid memory issues
durations = sampled_df['trip_duration_minutes']

# Freedman–Diaconis rule for bin width
q25, q75 = np.percentile(durations, [25, 75])
iqr = q75 - q25
n = len(durations)
bin_width = 2 * iqr / (n ** (1/3))
bin_count = int(np.ceil((durations.max() - durations.min()) / bin_width))

print(f"Suggested bin count: {bin_count}")


# Count how many trips exceed the cutoff
sampled_exceed = (sampled_df['trip_duration_minutes'] > cutoff).sum()
full_exceed = (trips_df['trip_duration_minutes'] > cutoff).sum()

print(f"Trips in sampled_df exceeding {cutoff} minutes: {sampled_exceed}")
print(f"Trips in trips_df exceeding {cutoff} minutes: {full_exceed}")


insights :
1. The massive bar near 0-20 minutes clearly shows that most bike trips are very short. This is typical for bike-sharing systems, often used for short commutes or quick errands.
2. The presence of bars, even if very short, extending all the way to 1440 minutes shoes that some trips in the data did take more then a day , the number of trips is 361





---
Task2
---



In [ ]:
# Use the original (not divided) trip durations
durations = sampled_df['trip_duration_minutes']
types = sampled_df['rideable_type']

# Build the box plot grouped by rideable_type
fig = go.Figure()

# Loop through each rideable type and add a box
for bike_type in sampled_df['rideable_type'].unique():
    fig.add_trace(go.Box(
        y=sampled_df[sampled_df['rideable_type'] == bike_type]['trip_duration_minutes'],
        name=bike_type,
        boxpoints='outliers',  # show outliers only
        marker_color='green',
        line_color='black',
        opacity=0.8
    ))

fig.update_layout(
    title="Box Plot of Trip Duration by Rideable Type",
    yaxis_title="Trip Duration (minutes)",
    xaxis_title="Rideable Type",
    template='simple_white'
)

# Render statically to avoid Colab issues
fig.show(config={'staticPlot': True})


insights :
1. both types show a very compact box near 20 minutes, indicating that the vast majority of trips for both bike types are quite short.
2. The median line is very close to the bottom of the box, confirming heavy right-skewness, this means almost all of the middle 50% of data is concentrated very close to the lower end, and the remaining data (up to Q3) is more spread out.
3. The green dots above the whiskers clearly represent the longer  trips with some of them above the 1440 line
4. Electric bikes seem to have a slightly tighter distribution,This suggests that while both have short typical trips, classic bikes might have a slightly wider range of trips durations
5. Both bike types exhibit very long duration "outliers," with classic bikes potentially having more extreme longer-duration outliers , It suggests that while electric bikes facilitate shorter, perhaps faster trips, classic bikes are used for the most extended journeys.
This could be due to factors like cost , battery limits, or simply user preference .



---
Task3
---



In [ ]:
trips_df['member_casual'].value_counts()

In [ ]:
urations = sampled_df['trip_duration_minutes']
types_to_group_by = sampled_df['member_casual']

fig = go.Figure()

for member_type in sampled_df['member_casual'].unique():
    fig.add_trace(go.Box(
        y=sampled_df[sampled_df['member_casual'] == member_type]['trip_duration_minutes'],
        name=member_type,
        boxpoints='outliers',
        marker_color='green',
        line_color='black',
        opacity=0.8
    ))

fig.update_layout(
    title="Box Plot of Trip Duration by Member Type",
    yaxis_title="Trip Duration (minutes)",
    xaxis_title="Member Type",
    template='simple_white'
)

# Render statically to avoid Colab issues
fig.show(config={'staticPlot': True})

insights :
1. As with the previous duration plots, both "casual" and "member" trips show an incredibly strong right-skewness. The box for both categories is extremely compact and squashed at the very bottom of the plot (close to 0 minutes), and the median line is practically on top of the first quartile (Q1). This reinforces that the vast majority of trips for both casual and member users are very short.
2. While both are low, the median (and Q1/Q3) for 'casual' riders appears slightly higher (or at least, the box is marginally less squashed) than for 'member' riders. This suggests that a "typical" trip for a casual user is slightly longer than for a member, even if both are still relatively short.
3. Both categories clearly have a significant number of "outlier" trips (the green dots) that extend far beyond the main box and whiskers, indicating that very long trips do occur for both types of users.
4. The box (IQR) for 'casual' riders seems marginally wider than for 'member' riders,The whiskers for 'casual' riders also appear slightly longer, This implies casual riders exhibit a greater range of typical trip durations compared to members.
5. 'Casual' riders have a significantly higher density of very long outlier trips, 'Member' riders also have long outlier trips, but they are fewer in number and generally do not reach the same extreme durations as those of casual riders.





---
Task4
---



In [ ]:
# Counting Trips Longer Than One Day
# Define threshold: 1 day = 1440 minutes
one_day_minutes = 1440
# Filter trips longer than 1 day
long_trips_df = trips_df[trips_df['trip_duration_minutes'] > one_day_minutes]
long_sampled_df = sampled_df[sampled_df['trip_duration_minutes'] > one_day_minutes]
# Show how many there are
print(f"Total number of trips longer than 1 day in full data: {len(long_trips_df)}")
print(f"Total number of trips longer than 1 day in sampled data: {len(long_sampled_df)}")
# Combine start and end station counts for long trips
start_counts = long_trips_df['start_station_id'].value_counts()
end_counts = long_trips_df['end_station_id'].value_counts()

# Combine them into a single Series
total_counts = start_counts.add(end_counts, fill_value=0).astype(int)

# Get station info: name and location
stations = sampled_df[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
stations = stations.rename(columns={
    'start_station_id': 'station_id',
    'start_station_name': 'station_name',
    'start_lat': 'lat',
    'start_lng': 'lng'
})

# Merge with counts
stations['long_trip_count'] = stations['station_id'].map(total_counts).fillna(0).astype(int)

# Filter stations with at least 1 long trip
stations = stations[stations['long_trip_count'] > 0]


In [ ]:
# Center the map on Washington DC
m = folium.Map(location=[38.9072, -77.0369], zoom_start=12, tiles='cartodbpositron')

# Optional: cluster points
marker_cluster = MarkerCluster().add_to(m)

# Add stations to the map
for _, row in stations.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=3 + row['long_trip_count']**0.5,  # scale marker size
        color='darkred',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.7,
        popup=f"{row['station_name']}<br>Trips > 1 day: {row['long_trip_count']}"
    ).add_to(marker_cluster)

# Show the map
m


insights :
1.  primary observation is that stations associated with long-duration trips are particularly concentrated around the central part WDC map ,This suggests that while very long trips are rare overall, they are not uniformly distributed but rather originate from or end in specific zones.
2. The 2496 is the count of unique station IDs that appear as either a start_station_id OR an end_station_id within those 361 trips.
This is a significant number of stations involved, considering that the total number of such trips was only 361. This means these long trips are spread across a wide variety of stations, rather than being concentrated at just a few specific locations.
3. This highlights areas where the system might need to adapt to different user behaviors like offering specific "long-term rental" options or different pricing for these stations.




---

# C)

---





---
Task1
---



In [ ]:
len(sampled_df['trip_cost'].unique())

trips_df.columns
len(sampled_df[sampled_df['start_month'] != sampled_df['end_month']])
sampled_df['start_time'] = pd.to_datetime(sampled_df['start_time'])

In [ ]:
import plotly.express as px

# cost Histogram
fig = px.histogram(sampled_df, x='trip_cost', nbins=141, title='distrupation of trips cost')
fig.show()

# cost Boxplot
fig = px.box(sampled_df, y='trip_cost', title='Boxplot of trips cost')
fig.show()


- نلاحظ ان اغلب الداتا متوزعة بين ال0 - وال10 دولار بكثرة وان القمة بين 3.5 و4 وهذا يدل على انه يوجد الكثير من الناس مشتركة واغلب الرحل لا تتجاوز ال45 دقيقة

- وايضا يوجد قيم اكبر صحيح انها نادرة ولكنها متوزعة وهذا يدل انه يوجد اشخاص تاخذها لمسافات كبيرة ولكنها قليلة  
- غالبا الرحل ذات تكلفة العالية اشخاص غير مشتركين بالاضافة الى انهم قد يكونون مرة واحدة فقط يستخدمون الدراجات ولا يعودون الى استخدامها بعد تجربة الخدمة ورؤية السعر




---
task2
---



In [ ]:
fig = px.scatter(sampled_df, x='trip_duration_minutes', y='trip_cost', trendline='ols',title='the realtion between duration and cost')
fig.show()
# lowess', 'rolling', 'ewm', 'expanding', 'ols'



*   النقاط التي قيمتها قريبة من الصفر كوقت هي تمثل الاعضاء التي لديهم اشتراك ولم يتجاوزوا ال45 دقيقة وكما نلاحظ هم كثر
*   ولدينا ثلاث توزعات للنقاط وذلك يعود بسبب الاشتراك او عدمه وحتى مروره بالمنطقة التجارية






---
Task3
---




In [ ]:
fig = px.scatter(sampled_df, x='temp', y='trip_cost', color='member_casual',
                 title='cost vs temperatur ')
fig.show()



*   اغلب الرحلات تكون بين 5 درجات وال20 درجة
*   عندما تكون درجة الحرارة فوق ال20 نلاحظ ان عدد الرحلات قليل
* كما نلاحظ اغلب رحل المشتركين الكلفة غالبا اقل من 10 دولار
* نلاحظ ان اغلب الكلف العالية من الغير المشتركين
* لايوجد علاقة واضحة بين درجة الحرارة والتكلفة لكن يمكن الفول ان بين ال 5 -15 يمكن للناس ان تذهب برحلات أطول






---
Task4
---




In [ ]:
daily_rev = sampled_df.groupby(sampled_df['start_time'].dt.date)['trip_cost'].sum().reset_index(name='revenue')
fig = px.line(daily_rev, x='start_time', y='revenue', title='daily incomes')
fig.show()

sampled_df['week'] = sampled_df['start_time'].dt.isocalendar().week
weekly_rev = sampled_df.groupby('week')['trip_cost'].sum().reset_index(name='revenue')
fig = px.line(weekly_rev, x='week', y='revenue', title='weekly incomes')
fig.show()



*   بالنسبة للايرادات اليومبة نلاحظ وجود بين هبوط وصعود ومع استمرار الايام نلاحظ زيادة بالدخل ونلاحظ تناوب بين صعود وهبوط في الايام ويعود هذا الامر اتوقع انو شخص يلي بيركب يوم بريح اليوم يلي بعدو

*   لدينا بشهر april هبوط واضح في الربح السبب قد يعود الى عدم وجود داتا كافية في هذا الشهر

* بالنسبة للايرادات الاسبوعية ملاحظ انه بشكل عام الامور نحو زيادة حيث ان هذا التذبذب راح بسبب انو الاسبوعي عطانا الشكل العام بالاسبوع فاصبح خط  اكثر انسيابية

* بشكل عام يوجد مشكلة في شهر april





---
Task5
---




In [ ]:
monthly_rev = sampled_df.groupby('start_month')['trip_cost'].mean().reset_index(name='avg_revenue')
fig = px.line(monthly_rev, x='start_month', y='avg_revenue', title='average month income')
fig.show()

* يبين لنا المخطط تطور متوسط تكلفة الرحلة الواحدة خلال شهر الاول كان متوسط تكلفة الرحلة ما يقارب 3.78 دولار مع دخول الشهر الثاني نلاحظ ارتفاع طفيف ويستمر الارتفاع بشكل طفيف حتى الشهر الثالث هذا النمو التدريجي يوحي بان شيئاً ما كان يتغير ببطء وثبات ربما كان المستخدمون يميلون لأخذ رحلات أطول قليلًا، أو أن هناك زيادة طفيفة في استخدام الدراجات ذات التكلفة الأعلى، أو ربما كان هناك تزايد في الرحلات التي تتخطى الحدود الزمنية المجانية للمشتركين وتتحمل رسومًا إضافية. هذه الزيادة، وإن كانت صغيرة، تشير إلى أن قيمة الرحلة الواحدة كانت في ازدياد
* ثم نصل الى شهر الرابع نلاحظ قفزة في متوسط الرحلة الواحدة بشكل ملحوظ حيث وصل ال4 دولار مسجل اعلى متوسط خلال هذه الفترة قد يبدو للحظة ان الامر جيد ولكن مع النظر الى مخطط اليومي والاسبوعي فنحدد شهدنا هبوط في هذا الشهر وقد يعود سبب الهبوط في رفع سعر الرحلة مما ادى الانهيار الخدمة انهياراً كارثياً

* وايضا ممكن هذا الارتفاع اتى بما انه عدد الرحلات الاجمالية في الشهر الرابع قليلة فوجود قيم شاذة او مرتفعة كما شهدنا في مخطط كلف الرحل سيرفع متوسط كلفة الرحلة بهذا الشكل


---

# **الخلاصة**


*   كانت خدمة مشاركة الدراجات تشهد نموًا مستمرًا في إجمالي إيراداتها وفي قيمة الرحلة الواحدة من يناير وحتى منتصف مارس.
*   مع ذلك، في أواخر مارس/أوائل أبريل، قد تكون الشركة تعرضت لحدث جسيم (إما إغلاق، أو تعليق، أو عطل كبير في النظام) أدى إلى توقف شبه كامل لجميع الرحلات والإيرادات, او قد يكون بسبب رفع رسوم الرحلة
* القفزة في متوسط تكلفة الرحلة في أبريل، على الرغم من أنها تبدو إيجابية في هذا الرسم البياني بمفرده، هي في الواقع مجرد انعكاس لحقيقة أن الرحلات القليلة جدًا المتبقية كانت هي الأكثر تكلفة، مما يلقي الضوء على الوضع الكارثي للخدمة في هذا الشهر.

* قد يكون سبب اخذ قرار الشركة برفع انها كانت تحاول رفع الرسوم في الاشهر الاولى ولكن بشكل طفيف وعندما وجدت ان المبيعات تزاد قامت بهذه الرفعة ظنا منها انه اصبح لديها قاعدة جماهيرية كبيرة وان المستخدمين بازدياد لتفاجئ بحصول عكس ذلك تماما
* كل هذه الامور هي مجرد تفسيرات ممكنة

* قد يكون سبب الارتفاع هو وجود تضخم




---

# D)

---





---
Task1
---




In [ ]:


# --- (Your existing data preparation for res_points) ---
# Assuming 'sampled_df' and 'Residential_Visitor_Parking_Zones' are loaded

# Step 0: Load residential zones GeoDataFrame
res_zones = Residential_Visitor_Parking_Zones
res_zones = res_zones.to_crs(epsg=4326)

# Step 1: Create GeoDataFrames for start and end points (using sampled_df)
start_gdf = gpd.GeoDataFrame(
    sampled_df,
    geometry=gpd.points_from_xy(sampled_df['start_lng'], sampled_df['start_lat']),
    crs='EPSG:4326'
)
end_gdf = gpd.GeoDataFrame(
    sampled_df,
    geometry=gpd.points_from_xy(sampled_df['end_lng'], sampled_df['end_lat']),
    crs='EPSG:4326'
)

# Step 2: Spatial join to check which points fall inside residential zones
start_in_res = gpd.sjoin(start_gdf, res_zones, predicate='within', how='inner')
end_in_res = gpd.sjoin(end_gdf, res_zones, predicate='within', how='inner')

# Step 3: Extract lat/lon of trips touching residential zones
res_start_points = start_in_res[['start_lat', 'start_lng']].rename(columns={'start_lat': 'lat', 'start_lng': 'lon'})
res_end_points = end_in_res[['end_lat', 'end_lng']].rename(columns={'end_lat': 'lat', 'end_lng': 'lon'})
res_points = pd.concat([res_start_points, res_end_points], ignore_index=True)

# --- Load CBD Data and ensure correct CRS (EPSG:4326) ---
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=4326) # Ensure CBD is also in EPSG:4326

# --- Create the Plotly Graph Object Figure ---
fig = go.Figure()

# Add the Heatmap Trace
fig.add_trace(go.Densitymapbox(
    lat=res_points['lat'],
    lon=res_points['lon'],
    radius=10, # Adjust radius as needed for visual effect
    colorscale="Viridis", # Or "Jet", "Hot", "Portland", etc. for heatmap colors
    hoverinfo="skip"
))

# Add Residential Zones as a GeoJSON layer
fig.update_layout(
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "geojson",
            "source": json.loads(res_zones.to_json()),
            "type": "line",
            # "line": {"width": 1, "color": "blue"} # Corrected: used "color" within "line" object
        },
        # Add CBD as another GeoJSON layer
        {
            "below": 'traces',
            "sourcetype": "geojson",
            "source": json.loads(CBD.to_json()),
            "type": "line",
            # "line": {"width": 2, "color": "green"} # Corrected: used "color" within "line" object
        }
    ]
)

# Update layout for map style, center, zoom, and title
fig.update_layout(
    title_text='Geographic Heatmap of Trips to Residential Zones with Borders',
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center = {"lat": res_points['lat'].mean(), "lon": res_points['lon'].mean()},
    margin={"r":0,"t":40,"l":0,"b":0},
)

# Render statically (Plotly handles this directly when fig.show() is called in a notebook)
fig.show(config={'staticPlot': True})

insights :
1. The most prominent insight is the very high density (bright yellow areas) of bike trips that overwhelmingly coincides with and largely respects the boundaries of the residential zones.
 This indicates that the bike-sharing system is heavily used for purposes directly related to residential areas, whether starting from, ending in, or traversing through them.
2. The highest concentration of trips within residential zones appears to be in the central and northern parts of Washington D.C., This suggests these are the most active residential areas for bike-sharing usage.
3. While the overall pattern is dense,These could indicate areas with fewer stations, different demographic profiles, or less need for bike-sharing, which could be further explored.



---
Task2
---



In [ ]:
# Step 1: Count trips per geohash sector
geohash_counts = sampled_df['geohash_p6'].value_counts().reset_index()
geohash_counts.columns = ['geohash_p6', 'trip_count']

# Optional: sort alphabetically or by count
geohash_counts = geohash_counts.sort_values(by='trip_count', ascending=False)

# Step 2: Plot
fig = px.bar(
    geohash_counts,
    x='geohash_p6',
    y='trip_count',
    title='Distribution of Trips by Geographic Sector (Geohash_p6)',
    labels={'geohash_p6': 'Geographic Sector', 'trip_count': 'Number of Trips'}
)

# Step 3: Turn off interactivity
fig.show(config={'staticPlot': True})


In [ ]:


# --- Your Function to Convert Geohash to Polygon ---
def geohash_to_polygon(geohash_str):
    """
    Converts a geohash string to a shapely Polygon representing its bounding box.
    """
    lat, lon, lat_err, lon_err = gh.decode_exactly(geohash_str)
    min_lat = lat - lat_err
    max_lat = lat + lat_err
    min_lon = lon - lon_err
    max_lon = lon + lon_err
    coords = [
        (min_lon, min_lat),
        (max_lon, min_lat),
        (max_lon, max_lat),
        (min_lon, max_lat),
        (min_lon, min_lat)  # Close the polygon
    ]
    return Polygon(coords)

# --- Sample Input: Top 5 Geohashes (replace with your real data) ---
top_5_geohashes = geohash_counts.head(5)['geohash_p6'].tolist()
geohash_polygons = [geohash_to_polygon(g) for g in top_5_geohashes]

# --- Convert to GeoDataFrame for easier plotting ---
gdf = gpd.GeoDataFrame(geometry=geohash_polygons, crs='EPSG:4326')

# --- Plot using Plotly ---
fig = go.Figure()

# Add each polygon as a filled scattermapbox trace
for poly in gdf.geometry:
    lons, lats = poly.exterior.coords.xy
    fig.add_trace(go.Scattermapbox(
        lon=list(lons),
        lat=list(lats),

        mode='lines',
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.4)',  # Semi-transparent red
        line=dict(width=1, color='red'),
        hoverinfo='skip',
        showlegend=False
    ))

# Map settings
fig.update_layout(
    mapbox_style='carto-positron',
    mapbox_zoom=12,
    mapbox_center={"lat": gdf.geometry.centroid.y.mean(), "lon": gdf.geometry.centroid.x.mean()},
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Static rendering (no interactions)
fig.show(config={'staticPlot': True})


insights :
1.  The first few bars on the left are significantly taller than the rest, with the tallest bar approaching 800 trips, while many others have less than 50. This indicates that bike-sharing activity is heavily concentrated in a few high-demand areas.
2. the plot allows for quick identification of the highest-demand geographic sectors. These are likely to correspond to areas with high population density, major transit hubs, popular attractions, or dense commercial/office districts
3. we ploted these top sectors and we compared it the heatmap in task1 , and we are now sure that they are in hotspots for trips in residential zone




---
Task3
---



In [ ]:

def freedman_diaconis_bins(data):
    data = data.dropna()
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    n = len(data)
    if n == 0 or iqr == 0:
        return 10  # fallback if data is empty or IQR is zero
    bin_width = 2 * iqr / (n ** (1/3))
    if bin_width == 0:
        return 10
    bin_count = int(np.ceil((data.max() - data.min()) / bin_width))
    print("Bin count : ", bin_count)
    return bin_count


# Calculate bins for each variable
bins_cbd = freedman_diaconis_bins(sampled_df['distance_to_cbd_m'])
bins_metro = freedman_diaconis_bins(sampled_df['start_nearest_metro_distance'])
bins_shuttle = freedman_diaconis_bins(sampled_df['start_nearest_shuttle_distance'])

sampled_df['distance_to_cbd_km'] = sampled_df['distance_to_cbd_m'] / 1000

# 1. Distance to CBD
fig1 = px.histogram(
    sampled_df,
    x='distance_to_cbd_km',
    nbins=bins_cbd,
    title='Distribution of Distance to CBD (km)',
    labels={'distance_to_cbd_km': 'Distance to CBD (km)'}
)
fig1.show(config={'staticPlot': True})

# 2. Closest Metro Station Distance
fig2 = px.histogram(
    sampled_df,
    x='start_nearest_metro_distance',
    nbins=bins_metro,
    title='Distribution of Distance to Nearest Metro Station',
    labels={'start_nearest_metro_distance': 'Distance to Metro (km)'}
)
fig2.show(config={'staticPlot': True})

# 3. Closest Shuttle Station Distance
fig3 = px.histogram(
    sampled_df,
    x='start_nearest_shuttle_distance',
    nbins=bins_shuttle,
    title='Distribution of Distance to Nearest Shuttle Station',
    labels={'start_nearest_shuttle_distance': 'Distance to Shuttle (km)'}
)
fig3.show(config={'staticPlot': True})


insights :
1. The overwhelming majority of trips (as indicated by the highest bars) occur within approximately 0 to 4 kilometers of the Central Business District. The peak frequency appears to be around 2.2 kilometers, This reinforces the CBD's role as a major hub for bike-sharing activity.
2. this strongly supports the idea that the bike-sharing system primarily serves "last-mile" transportation, short-distance commutes, or intra-CBD movement.


---


1. The histogram displays a highly skewed distribution, with an extremely high frequency of trips originating or ending very close to Metro stations. The counts drop off sharply as the distance increases.
2. This insight highlights the importance of placing bike-sharing stations in very close proximity to Metro entrances to maximize their integration with the public transit network and serve commuter needs effectively.


---
1.  Similar to the Metro station plot, this histogram shows a highly skewed distribution, with the highest frequency of trips occurring very close to shuttle stations.
2. This suggests a consistent user behavior of using bike-sharing to bridge short gaps to fixed public transport points.




---
Task4
---



In [ ]:

# Categorize trips
def classify_trip(row):
    if row['start_in_cbd'] == 1 and row['end_in_cbd'] == 1:
        return 'Fully in CBD'
    else:
        return 'Outside CBD'

# Apply classification
sampled_df['cbd_trip_type'] = sampled_df.apply(classify_trip, axis=1)

# Count
trip_cbd_counts = sampled_df['cbd_trip_type'].value_counts().reset_index()
trip_cbd_counts.columns = ['Trip Type', 'Count']

# Plot
fig = px.bar(
    trip_cbd_counts,
    x='Trip Type',
    y='Count',
    title='Trips Fully in CBD vs Outside',
    text='Count',
    labels={'Count': 'Number of Trips'}
)

fig.update_traces(textposition='outside')
fig.update_layout(yaxis_title='Number of Trips', xaxis_title='Trip Category')
fig.show(config={'staticPlot': True})


In [ ]:
trips_df['cbd_trip_type'] = trips_df.apply(classify_trip, axis=1)
full_trip_cbd_counts = trips_df['cbd_trip_type'].value_counts().reset_index()
full_trip_cbd_counts.columns = ['Trip Type', 'Count']
full_trip_cbd_counts['Percentage'] = (full_trip_cbd_counts['Count'] / full_trip_cbd_counts['Count'].sum()) * 100
full_trip_cbd_counts

insights :
1. The vast majority of trips  fall into the "Outside CBD" category, This indicates that the bike-sharing system primarily serves a broader geographical area beyond just the core CBD, or facilitates connections into/out of it rather than exclusively within it.
3.  A significantly smaller number of trips (1,243 trips) were classified as "Fully in CBD". This suggests that while there is some intra-CBD movement, it represents a much smaller proportion
4. This distribution implies that the bike-sharing service's primary function might not be solely for quick hops within the CBD itself, but rather for facilitating commutes, errands, or leisure travel that connects to or traverses the CBD from other parts of the city. This aligns with the "first/last mile" insights from the distance histograms, where trips are often connecting to major transit points or residential areas that might lie outside the strict CBD boundaries.
5.  The fact that over 93% of all the trips involve areas outside the CBD strongly reinforces the idea that bike-sharing is primarily used as a "first/last mile" solution, connecting users to or from transit hubs (like Metro and Shuttle stations) or residential areas that often lie outside the strict CBD boundaries. These trips are unlikely to be exclusively within the CBD.



---
Task5
---



In [ ]:

# Filter trips that passed through CBD
cbd_passed_df = sampled_df[
    (sampled_df['start_in_cbd'] == 1) | (sampled_df['end_in_cbd'] == 1)
]

# Group by rideable_type and member_casual
grouped = cbd_passed_df.groupby(['rideable_type', 'member_casual']).size().reset_index(name='trip_count')

# Plot
fig = px.bar(
    grouped,
    x='rideable_type',
    y='trip_count',
    color='member_casual',
    barmode='group',
    title='Trips That Passed Through CBD by Rideable Type and Membership',
    labels={'trip_count': 'Number of Trips', 'rideable_type': 'Bike Type'}
)

fig.update_layout(
    xaxis_title='Rideable Type',
    yaxis_title='Number of Trips'
)
fig.show(config={'staticPlot': True})


In [ ]:
cbd_passed_df_trips_df=trips_df[
    (trips_df['start_in_cbd'] == 1) | (trips_df['end_in_cbd'] == 1)
]

# Group by rideable_type and member_casual
grouped = cbd_passed_df_trips_df.groupby(['rideable_type', 'member_casual']).size().reset_index(name='trip_count')

print(f"Length of cbd_passed_df_trips_df: {len(cbd_passed_df_trips_df)}")
print(f"Length of trips_df: {len(trips_df)}")

percentage = (len(cbd_passed_df_trips_df) / len(trips_df)) * 100
print(f"Percentage of cbd_passed_df_trips_df compared to trips_df: {percentage:.2f}%")

insights :
1. the extra exploration shows that 29.85% of all bike-sharing trips pass through the CBD, This is a substantial portion, indicating the CBD's critical role as an origin, destination, or transit point for a large segment of bike-sharing users.
2. For both classic_bike and electric_bike categories, members (orange/red bars) consistently account for a significantly higher number of trips that pass through the CBD compared to casual riders (blue bars). This suggests that regular commuters or frequent users (members) are more likely to utilize bike-sharing for travel involving the city's central business district.
3. Both casual and member riders use classic_bike more frequently for CBD-involved trips than electric_bike
4. Since members are the primary users for CBD-involved trips, strategies to retain and grow membership, and ensure sufficient bike availability in and around the CBD, are crucial.
5. The demand for both classic and electric bikes for CBD-involved trips means that fleet management should consider a balanced distribution to meet the preferences of both member and casual riders.




---
Task6
---



In [ ]:
# Create a contingency table
# (Counts of each combination)
# Make sure we’re using categorical data
contingency_table = pd.crosstab(trips_df['close_to_cbd'], trips_df['member_casual'])
contingency_table


In [ ]:
# Run chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi2 Statistic:", chi2)
print("Degrees of Freedom:", dof)
print("P-value:", p)
# interpretion based on the p-value:
if p < 0.05:
    print(" There is a significant correlation between distance to CBD segments and membership type.")
else:
    print(" No significant correlation found between distance to CBD segments and membership type.")


| α Value  | Interpretation                                                                |
| -------- | ----------------------------------------------------------------------------- |
| **0.05** | Most common — means you're willing to accept a 5% chance of a false positive. |
| 0.01     | Stricter — used in more critical fields (medicine, etc.).                     |
| 0.10     | Looser — sometimes used in exploratory research.                              |


insights:
1. The p-value of 0.0, which is much less than the conventional significance level of 0.05 leads us to believe there is a statistically significant relationship between whether a trip is close to the CBD and the user's membership type.
2. While the Chi-square test indicates strong statistical significance (due to the very large sample size), the proportional difference between casual and member trips being close_to_cbd is relatively small,This suggests thatBoth member and casual riders have roughly half their trips originating or ending close to the CBD.



---
# E)
---


---
Task 1
---

In [ ]:
sampled_df.columns

In [ ]:
sampled_df['rideable_type'].unique()

In [ ]:
daily_weather_avg = sampled_df.groupby('date')[['temp', 'humidity', 'windspeed']].mean().reset_index()
daily_weather_avg = daily_weather_avg.rename(columns={
    'temp': 'Average Temperature',
    'humidity': 'Average Humidity',
    'windspeed': 'Average Wind Speed'
})
fig = px.line(
    daily_weather_avg,
    x='date',
    y=['Average Temperature', 'Average Humidity', 'Average Wind Speed'], # List of columns for y-axis
    title='Average Daily Weather Conditions (Temperature, Humidity, Wind Speed)',
    labels={
        'date': 'Date',
        'value': 'Average Value', # Default label for the combined y-axis values
        'variable': 'Metric'     # Default label for the legend (which variable is which line)
    }
)
fig.update_layout(hovermode="x unified") # Enhances hover tooltips for multiple lines
fig.show()



---
Task2
---

In [ ]:
daily_weather_cond = sampled_df.groupby('date')['weather_segment'].first().reset_index()

daily_rev = sampled_df.groupby(sampled_df['date'])['trip_cost'].sum().reset_index(name='revenue')

merged_df = pd.merge(daily_rev, daily_weather_cond, on='date', how='left')
fig = px.box(
    merged_df,
    x='weather_segment',  # Categorical variable on x-axis
    y='revenue',      # Numerical variable on y-axis
    title='Daily Revenue by Weather Condition',
    labels={
        'weather_condition': 'Weather Condition',
        'daily_revenue': 'Daily Revenue ($)'
    },
    category_orders={"weather_condition": ["Sunny", "Cloudy", "Rainy"]} # Optional: ensure specific order
)
fig.update_traces(boxpoints='all', jitter=0.3) # Show individual points for more detail

fig.show()



*   نلاحظ بالنسبة للايام الماطرة يقع وسطيا الايرادات عند مايقارب 650 دولار وهو  اقل  متوسط من جميع حالات طقس رغم وجود عدد كبير من الايام ماطرة ما يقارب 55 بالمئة من الأيام هي ماطرة ونلاحظ مدى توسع الصندوق وهذا يشير الى تقلب كبير في الايرادات في الأيام الماطرة وملاحظ هذا حيث لدينا ايام الايرادات تقارب الصفر وبعض متجاوزة الالف واعتقد يعود السبب الى القيم القريبة الى الصفر هي الايام ذو امطار شديدة وهذا منطقي من الصعب عندها ركوب الدراجات اما بالنسبة للقيم العالية وارد ان بعض الايام الممطرة تكون مقبولة وهذا يعود الى بعض انواع المستخدمين التي تستمع في ذلك او بسبب الحاجة العاجلة للدراجة بدل الانتظار
*   نلاحظ بالنسبة للايام الغائمة مرتفع وسطي الايرادات لما يقارب 800 دولار اكثر من الايام الماطرة ونلاحظ انه يوجد استقرار وليس تقلب بالايرادات وايضا الاتجاه الايرادات في ايام الغائمة اما بازدياد او استقرار ونلاحظ قفزات عالية جدا في الايرادات وارد ذلك عند درجات الحرارة المعتدلة اما بالنسبة للقيم المتدنية جدا فهي اما بشهر الرابع او انها كانت ايام عطل

* رغم قلة الايام المشمسة الا اننا نجد ان الناس تتجه لاستخدام الدراجات وهي اعلى متوسط دخل وملاحظ ان الناس في الايام المشمسة تميل الى استخدام الدراجات وقد يعود ذلك بسبب قلة الايام المشمسة الموجود فالناس تحب التعرض للشمس لذلك تفضل عندها استخدام الدراجات بالاضافة ان الجو يكون جيد

* حيث نستنتج تأثير الطقس على سلوك الركاب يظهر بوضوح كيف تؤثر حالة الطقس بشكل مباشر على الإيرادات اليومية، حيث يفضل الناس استخدام الدراجات في الطقس المعتدل والمشمس، مما يؤدي إلى زيادة الإيرادات، في حين أن الأيام الممطرة العكس اقل مستخدمين وايرادات اقل

* لكن وجود القيم الشاذة في جميع الفئات يدل على أن هناك دائمًا بعض الأيام التي لا تتبع النمط العام للطقس، سواء كانت جيدة بشكل استثنائي أو سيئة بشكل استثنائي حيث عندها اتوقع يوجد امور اخرى  تدخل عندها



---
Task3
---

In [ ]:
# lowess', 'rolling', 'ewm', 'expanding', 'ols'
# --- Apply Min-Max Normalization to 'daily_revenue' ---
# xi-xmin /xmax-xmin

# min_revenue = daily_rev['revenue'].min()
# max_revenue = daily_rev['revenue'].max()
# daily_rev['normalized_daily_revenue'] = (daily_rev['revenue'] - min_revenue) / (max_revenue - min_revenue)

merg = pd.merge(daily_weather_avg,daily_rev,on='date',how='left')

cols_to_normalize = ['revenue', 'Average Temperature', 'Average Humidity']
for col in cols_to_normalize:
    min_val = merg[col].min()
    max_val = merg[col].max()
    # Avoid division by zero if all values are the same
    if (max_val - min_val) != 0:
        merg[f'normalized_{col}'] = (merg[col] - min_val) / (max_val - min_val)
    else: # If all values are the same, normalized value is 0 (or 1, depends on convention)
        merg[f'normalized_{col}'] = 0.0

fig1 = px.scatter(merg,x='normalized_Average Temperature',y='normalized_revenue',
                 title="relationship between daily income and temperature",trendline='ols',
                 labels={
                   'Temperature': 'normalized_Average Daily Temperature',
                   'daily_revenue': 'Daily Revenue ($)' }
                 )
fig1.show()


fig2 = px.scatter(merg,x='normalized_Average Humidity',y='normalized_revenue',
                 title="relationship between daily income and Humidity",trendline='ols',
                 labels={
                   'Humidity': 'normalized_Average Daily humidity',
                   'daily_revenue': 'Daily Revenue ($)' }
                 )
fig2.show()


* بالنسبة للعلاقة بين الايرادات اليومية ودرجة الحرارة نلاحظ وجود علافة ارتباط خطي ايجابية حيث في درجات الحرارة المنخفضة (-5 - 3) نرى انخفاض في الايرادات ثم مع ازدياد درجة الحرارة نلاحظ انها تزداد الايردات الى ان تصل الى حد معين ثم تبدء بالنزول حيث ازدياد درجة الحرارة الى درجة ما وهي 16 يؤدي ازدياد الايرادات ولكن بعدها نرى ان ازدياد درجة الحرارة سيؤدي الى انخفاض في الايرادات


* بالنسبة للارتباط الخطي بين الايرادات والرطوبة لا يوجد علاقة ارتباط خطي حيث نلاحظ عند رطوبة منخقضة لدينا ايردادات مرتفعة وايرادات ومنخفضة والامر على القيم اي عندما تكون الرطوبة متوسطة او حتى عالية لدينا الايرادات مرات تكون منخفضة ومرات تكون عالية

---
 Task4
---

In [ ]:
  # 1. Create the Contingency Table
# This table shows the observed frequencies (counts) of each unique combination
# of weather segment and ride type.
# Rows: weather_segment
# Columns: rideable_type
contingency_table = pd.crosstab(sampled_df['weather_segment'], sampled_df['rideable_type'])
print("Contingency Table (Observed Frequencies):")
print(contingency_table)
print("\n" + "="*50 + "\n") # Visual separator in output

# 2. Perform the Chi-Square Test
# The chi2_contingency function performs the statistical calculations.
# It returns four values:
#   - chi2: The calculated Chi-Square statistic.
#   - p_value: The probability value (most important for interpretation).
#   - dof: Degrees of freedom.
#   - expected_frequencies: A 2D array of expected frequencies if the variables were independent.
chi2, p_value, dof, expected_frequencies = chi2_contingency(contingency_table)

print(f"Chi2 Statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of Freedom: {dof}")
print("\nExpected Frequencies Table:")

# Display the expected frequencies array as a DataFrame for better readability,
# using the same indices (rows) and columns as the observed contingency table.
print(pd.DataFrame(expected_frequencies, index=contingency_table.index, columns=contingency_table.columns))
print("\n" + "="*50 + "\n") # Another visual separator


# 3. Interpret the Results
# Define the significance level (alpha), which is the threshold for comparing the p_value.
# A common alpha level is 0.05 (or 5%).
alpha = 0.05
print("Interpretation of Results:")
if p_value < alpha:
    # If the p-value is less than alpha, we reject the null hypothesis.
    # The null hypothesis (H0) here is: There is no relationship between weather condition and ride type.
    print(f"Since the P-value ({p_value:.4f}) is less than the significance level (alpha = {alpha}),")
    print("we reject the null hypothesis (H0).")
    print("Conclusion: There is strong statistical evidence of a significant relationship between weather condition and ride type.")
    print("In other words, it appears that the distribution of ride types (or bike types) differs depending on the weather condition.")
    print("\n* To understand this relationship further, compare the observed frequencies with the expected frequencies to identify which categories contribute most to the association.")
else:
    # If the p-value is greater than or equal to alpha, we fail to reject the null hypothesis.
    print(f"Since the P-value ({p_value:.4f}) is greater than or equal to the significance level (alpha = {alpha}),")
    print("we fail to reject the null hypothesis (H0).")
    print("Conclusion: There is no sufficient statistical evidence to claim a significant relationship between weather condition and ride type.")
    print("In other words, it appears that the choice of ride type (or bike type) is not significantly affected by the weather condition, or any observed differences could be due to random chance.")


df_plot = contingency_table.reset_index().melt(id_vars='weather_segment', var_name='rideable_type', value_name='Count')

# 2. Draw a Grouped Bar Chart
fig = px.bar(
    df_plot,
    x='weather_segment',  # X-axis will be weather conditions
    y='Count',            # Y-axis will be the number of rides
    color='rideable_type',    # Different bars for each ride type within each weather condition
    barmode='group',      # This makes the bars for each ride_type stand side-by-side
    title='Ride Type Distribution by Weather Condition',
    labels={
        'weather_segment': 'Weather Condition',
        'Count': 'Number of Rides',
        'Ride Type': 'Ride Type'
    },
    category_orders={"weather_segment": ["Sunny", "Cloudy", "Rainy"]} # Optional: ensure specific order
)

fig.update_layout(xaxis_title="Weather Condition", yaxis_title="Number of Rides")
fig.show()


# Catching patterns

## A ) Temporal Analysis


### Task 1: Data Verification and Temporal Repairs
#### What to do:

- Verify that the temporal data you obtained is in the correct chronological order and without any missing intervals
- Perform necessary repairs for any temporal issues found

### Task 2: Future Revenue Predictions (15-day forecast)
#### What to do:

- Build a baseline model to predict future revenues for the next 15 days
- Use the method you find most appropriate for time series forecasting

### Task 3: Prophet Model Implementation
#### What to do:

- Use Facebook's Prophet library to model temporal time series components
- Use Prophet for future predictions and perform hyperparameter tuning for better model performance

### Task 4: Model Comparison Using Appropriate Evaluation Metrics
#### What to do:

- Compare your baseline model with Prophet model
- Use appropriate evaluation metrics for time series forecasting

### Task 5: Insights and Conclusions
#### What to conclude:

What will you learn from each component?

Analysis points:

Trend Analysis: Is bike usage increasing, decreasing, or stable over time?
Seasonality Patterns: Are there daily, weekly, or monthly patterns?
Model Performance: Which model performs better for forecasting?
Business Insights: What do the predictions tell us about future demand?


## B ) **General Analysis of Usage Patterns**

### Task 1: Data Sampling and Analysis
#### What to do:

- Perform sampling on the data to a degree where we can analyze and process it
- Make the samples comprehensive so that they represent the characteristics we see in the way that suits us

In [ ]:

# Sampled data is stored in `sampled_df`
print("Sampling already performed.")
print(f"Full data size: {len(trips_df)}")
print(f"Sampled data size: {len(sampled_df)}")
print("\nSampled data head:")
print(sampled_df.head())

# Basic statistical analysis on the sampled data to show its characteristics
print("\nSampled Data Descriptive Statistics:")
print(sampled_df.describe())

print("\nValue counts for key categorical columns in sampled data:")
print(sampled_df['member_casual'].value_counts())
print(sampled_df['rideable_type'].value_counts())
print(sampled_df['weather_segment'].value_counts())
print(sampled_df['kmeans_segment'].value_counts()) # Or 'volume_segment'

# Check if the sample distribution resembles the full dataset distribution for a few columns
print("\nComparison of value counts (Sampled vs Full Data):")
print("\nMember/Casual:")
print("Sampled:\n", sampled_df['member_casual'].value_counts(normalize=True))
print("Full:\n", trips_df['member_casual'].value_counts(normalize=True))

print("\nRideable Type:")
print("Sampled:\n", sampled_df['rideable_type'].value_counts(normalize=True))
print("Full:\n", trips_df['rideable_type'].value_counts(normalize=True))

print("\nTrip Cost Distribution (Sampled vs Full Data):")
print("Sampled:\n", sampled_df['trip_cost'].describe())
print("Full:\n", trips_df['trip_cost'].describe())

print("\nTrip Duration Distribution (Sampled vs Full Data):")
print("Sampled:\n", sampled_df['trip_duration_minutes'].describe())
print("Full:\n", trips_df['trip_duration_minutes'].describe())

print("\nSampling is comprehensive as the value counts and descriptive statistics for key columns in the sampled data are similar to the full dataset, indicating it represents the main characteristics.")


### Task 2: Clustering Analysis with Machine Learning
#### What to do:

- Perform clustering using machine learning techniques (three algorithms)
- Focus on the **two most important features**
Then describe and compare the clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

# Based on the previous analysis and the problem context, two important features could be:
# 1. trip_duration_minutes: Represents the usage length and potentially trip purpose.
# 2. trip_cost: Represents the value of the trip and is directly related to duration and type/location.
# Other potentially important features could be location-based distances, but duration and cost seem most direct for usage patterns.

features_for_clustering = sampled_df[['trip_duration_minutes', 'trip_cost']].dropna()

# Handle potential zero costs if they significantly skew data (though cleaning already addressed negative/high outliers)
features_for_clustering = features_for_clustering[features_for_clustering['trip_cost'] > 0]

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_clustering)



#### Clustering Algorithms

---




In [ ]:

# 1. K-Means
# Determine optimal k using the Elbow method (optional but good practice)
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10) # Explicitly set n_init
    kmeans.fit(scaled_features)
    inertia.append(kmeans.inertia_)

# Plot Elbow method (using Matplotlib)
plt.figure(figsize=(8, 4))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for K-Means')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.show()

# Let's choose k based on a plausible interpretation (e.g., k=3 or k=4 could be reasonable)
# Let's try k=3 for simplicity as a starting point
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # Explicitly set n_init
sampled_df.loc[features_for_clustering.index, 'kmeans_cluster'] = kmeans.fit_predict(scaled_features)



# For K-Means and Agglomerative Clustering, we can look at the mean of the features in each cluster.
# DBSCAN has a noise cluster (-1), so we'll treat it separately.

print("\n--- K-Means Cluster Analysis ---")
kmeans_summary = sampled_df.groupby('kmeans_cluster')[['trip_duration_minutes', 'trip_cost']].mean()
print(kmeans_summary)

# Visualize K-Means clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sampled_df, x='trip_duration_minutes', y='trip_cost', hue='kmeans_cluster', palette='viridis', legend='full')
plt.title('K-Means Clusters (Trip Duration vs Trip Cost)')
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Trip Cost')
plt.show()


print("\n--- Agglomerative Clustering Analysis ---")
agg_summary = sampled_df.groupby('agg_cluster')[['trip_duration_minutes', 'trip_cost']].mean()
print(agg_summary)


In [ ]:

# 2. Agglomerative Clustering (Hierarchical)
# We need to choose the number of clusters 'n_clusters'
agg_clustering = AgglomerativeClustering(n_clusters=k) # Using the same k as KMeans for comparison
sampled_df.loc[features_for_clustering.index, 'agg_cluster'] = agg_clustering.fit_predict(scaled_features)

# Visualize Agglomerative clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sampled_df, x='trip_duration_minutes', y='trip_cost', hue='agg_cluster', palette='viridis', legend='full')
plt.title('Agglomerative Clusters (Trip Duration vs Trip Cost)')
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Trip Cost')
plt.show()


print("\n--- DBSCAN Cluster Analysis ---")
# DBSCAN clusters and noise points
dbscan_summary = sampled_df.groupby('dbscan_cluster')[['trip_duration_minutes', 'trip_cost']].agg(['mean', 'count'])
print(dbscan_summary)


In [ ]:

# 3. DBSCAN
# DBSCAN does not require specifying the number of clusters beforehand
# It requires epsilon (eps) and minimum samples (min_samples)
# Estimating eps can be tricky; k-distance plot is common, but let's try some values.
# min_samples is typically set to 2*ndim or more.
# We'll use a relatively small eps and min_samples for this example.
dbscan = DBSCAN(eps=0.5, min_samples=5) # Adjust eps and min_samples based on data scaling and density
sampled_df.loc[features_for_clustering.index, 'dbscan_cluster'] = dbscan.fit_predict(scaled_features) # -1 is noise


# Visualize DBSCAN clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sampled_df, x='trip_duration_minutes', y='trip_cost', hue='dbscan_cluster', palette='viridis', legend='full')
plt.title('DBSCAN Clusters (Trip Duration vs Trip Cost)')
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Trip Cost')
plt.show()




#### Cluster Comparison

##### K-Means Cluster Means:
*[Display your kmeans_summary data here]*

##### Agglomerative Cluster Means:
*[Display your agg_summary data here]*

##### DBSCAN Cluster Means and Counts:
*[Display your dbscan_summary data here]*

##### Description and Comparison:

###### K-Means and Agglomerative Clustering
- K-Means and Agglomerative Clustering partition the data into a fixed number of clusters (k=3).
- Both algorithms identify clusters with distinct means for trip duration and cost.
- Cluster 0 in K-Means and Agglomerative likely represents short, low-cost trips.
- Cluster 1 or 2 in both likely represent longer, higher-cost trips.
- The exact mapping of cluster labels might differ between K-Means and Agglomerative for the same underlying cluster.
- K-Means is generally faster for large datasets and produces spherical clusters.
- Agglomerative Clustering can identify clusters of different shapes but is slower.

###### DBSCAN
- DBSCAN identifies dense regions as clusters and marks points in low-density regions as noise (-1).
- The number of clusters in DBSCAN is not fixed and depends on 'eps' and 'min_samples'.
- A significant number of points might be classified as noise if 'eps' is too small or 'min_samples' too high for sparser areas.
- DBSCAN is good at finding arbitrarily shaped clusters and is robust to outliers (which become noise points).
- Tuning 'eps' and 'min_samples' is crucial for DBSCAN's performance and the resulting clusters.

##### Overall Analysis

Using 'trip_duration_minutes' and 'trip_cost' features, clustering helps segment trip data into groups like 'short/cheap', 'medium/moderate', and 'long/expensive'.

K-Means and Agglomerative provide a fixed partition, while DBSCAN highlights dense usage patterns and identifies unusual trips as noise.

The choice of algorithm depends on whether a fixed number of segments is needed (K-Means, Agglomerative) or if identifying core usage areas and outliers is important (DBSCAN).

### Task 3: Post-Processing Operations
What to do:

- Perform operations after the appropriate processing to determine the most important features that had an impact
- Choose the best model for each algorithm that we used and compare them with a clear baseline that we initially built

### Task 4: Comprehensive Analysis Summary
#### What to conclude:
- For each clustering result, analyze and interpret:

#### Key Insights to Extract:

Cluster Interpretation:
python# Example interpretations:
```
# Cluster 0: Short trips during rush hours (commuters)
# Cluster 1: Long trips during leisure hours (tourists/recreation)
# Cluster 2: Medium trips throughout the day (mixed usage)
```
Business Recommendations:

Which user segments need different pricing strategies?
When should bike availability be optimized?
What are the peak usage patterns?


Algorithm Comparison:

Which algorithm captured the most meaningful patterns?
How do the clustering results differ between algorithms?
Which approach is most suitable for this type of data?



Additional Analysis You Can Include:
python# Seasonal patterns within clusters
sampled_df['month'] = sampled_df['started_at'].dt.month
sampled_df['day_of_week'] = sampled_df['started_at'].dt.dayofweek
```
# Add more features for deeper analysis
# Weather correlation, station popularity, etc.
```
Important Note:
Document why you chose specific features for clustering
Validate that your clusters make business sense, not just statistical sense